영업팀 일일보고 자동화 프로그램
===

- 프로그램 제작 목적
    - 인력으로는 매일 꼬박 30분이상 걸리는 작업을 자동화한다.
        - BeFore : 30분 이상 소요
        - After : 40 ~ 50 초 소요
    - 자료 기입 중 실수방지
        - 인력으로 진행하고 단계가 복잡해서 실수의 여지가 꽤 많았다.
    - 작업자별 차이 제거
        - 월 단위 로테이션으로 작업자를 바꿔서 작성한다.
        - 프로그램 제작 중에 알게되었지만 각자 집계기준을 서로 다르게 알고 있다.
            - 수치에 거의 차이가 없으면 상관없으나 차이가 있다.
        - 이 기회에 데이터 집계기준도 재정비하여 통일했다.

- 이 프로그램은 실행파일을 제작해서 영업팀에 배포했으므로 readme.md 파일을 참고할 것

- 브랜치 버전이 존재한다.
    - 최종버전 : sqlalchemy, xlwings 조합
    - pyautogui, win32com, openpyxl 등을 이용한 버전이 있다.
    - 작동에 이상은 없으나 시간단축이 불가능한 단계를 포함하면 자동화해도 10여분 정도 걸린다.

## ERP에서 자료 가져오기

In [1]:
from sqlalchemy import create_engine
import pandas as pd
import binascii   # 한글 변환에 필요한 라이브러리
import sys
from datetime import datetime, date, timedelta
from timeit import default_timer as timer
import pyautogui
from typing import List
from typing import Union

### DB 접속

In [2]:
sys.path.append('/settings')
import config

def connect_db(sid: str) -> 'sqlalchemy.engine.base.Engine':
    if sid != config.COMPANY_DB_CONFIG['sid']:
        raise ValueError("DB 를 찾을 수 없습니다.")
    
    conn = create_engine(
        "oracle+cx_oracle://{user}:{password}@{host}:{port}/{sid}?encoding=UTF-8&nencoding=UTF-8".format(
            user=config.COMPANY_DB_CONFIG['user'],
            password=config.COMPANY_DB_CONFIG['password'],
            host=config.COMPANY_DB_CONFIG['host'],
            port=config.COMPANY_DB_CONFIG['port'],
            sid=config.COMPANY_DB_CONFIG['sid']
        )
    )
    return conn

engine = connect_db('IVY')

now_version: str = 'v0.3' # 버전표기

#### 사용할 변수 입력

##### N 시즌 입력

In [3]:
N_season: str = pyautogui.prompt(title=f'N시즌_{now_version}', text='N시즌을 입력하세요. 예 : 23N', default='23N')

In [4]:
while True:
    i: int = 0
    if len(N_season) != 3:
        i += 1
        pyautogui.alert(title='입력오류', text='시즌은 3자리로 입력하세요')
        N_season = pyautogui.prompt(title='N시즌', text='N시즌을 입력하세요. 예 : 23N', default='23N')

    if N_season[-1] != 'N':
        i += 1
        pyautogui.alert(title='입력오류', text='마지막 글자는 N 입니다.')
        N_season = pyautogui.prompt(title='N시즌', text='N시즌을 입력하세요. 예 : 23N', default='23N')

    if (N_season[0] >= '0' and N_season[0] <= '9') == False:
        i += 1
        pyautogui.alert(title='입력오류', text='첫번째 문자는 숫자입니다.')
        N_season = pyautogui.prompt(title='N시즌', text='N시즌을 입력하세요. 예 : 23N', default='23N')
    
    if (N_season[1] >= '0' and N_season[1] <= '9') == False:
        i += 1
        pyautogui.alert(title='입력오류', text='두번째 문자는 숫자입니다.')
        N_season = pyautogui.prompt(title='N시즌', text='N시즌을 입력하세요. 예 : 23N', default='23N')

    if i == 0:
        break

##### F 시즌 입력

In [5]:
F_season: str = pyautogui.prompt(title='F시즌', text='F시즌을 입력하세요. 예 : 22F', default='22F')

In [6]:
while True:
    i: int = 0
    if len(F_season) != 3:
        i += 1
        pyautogui.alert(title='입력오류', text='시즌은 3자리로 입력하세요')
        F_season = pyautogui.prompt(title='F시즌', text='F시즌을 입력하세요. 예 : 22F', default='22F')

    if F_season[-1] != 'F':
        i += 1
        pyautogui.alert(title='입력오류', text='마지막 글자는 F 입니다.')
        F_season = pyautogui.prompt(title='F시즌', text='F시즌을 입력하세요. 예 : 22F', default='22F')

    if (F_season[0] >= '0' and F_season[0] <= '9') == False:
        i += 1
        pyautogui.alert(title='입력오류', text='첫번째 문자는 숫자입니다.')
        F_season = pyautogui.prompt(title='F시즌', text='F시즌을 입력하세요. 예 : 22F', default='22F')
    
    if (F_season[1] >= '0' and F_season[1] <= '9') == False:
        i += 1
        pyautogui.alert(title='입력오류', text='두번째 문자는 숫자입니다.')
        F_season = pyautogui.prompt(title='F시즌', text='F시즌을 입력하세요. 예 : 22F', default='22F')

    if i == 0:
        break 

##### S 시즌 입력

In [7]:
S_season: str = pyautogui.prompt(title='S시즌', text='S시즌을 입력하세요. 예 : 23S', default='23S')

In [8]:
while True:
    i: int = 0
    if len(S_season) != 3:
        i += 1
        pyautogui.alert(title='입력오류', text='시즌은 3자리로 입력하세요')
        S_season = pyautogui.prompt(title='S시즌', text='S시즌을 입력하세요. 예 : 23S', default='23S')

    if S_season[-1] != 'S':
        i += 1
        pyautogui.alert(title='입력오류', text='마지막 글자는 S 입니다.')
        S_season = pyautogui.prompt(title='S시즌', text='S시즌을 입력하세요. 예 : 23S', default='23S')

    if (S_season[0] >= '0' and S_season[0] <= '9') == False:
        i += 1
        pyautogui.alert(title='입력오류', text='첫번째 문자는 숫자입니다.')
        S_season = pyautogui.prompt(title='S시즌', text='S시즌을 입력하세요. 예 : 23S', default='23S')
    
    if (S_season[1] >= '0' and S_season[1] <= '9') == False:
        i += 1
        pyautogui.alert(title='입력오류', text='두번째 문자는 숫자입니다.')
        S_season = pyautogui.prompt(title='S시즌', text='S시즌을 입력하세요. 예 : 23S', default='23S')

    if i == 0:
        break 

##### 자료 작성기준일 입력

In [9]:
today: str = datetime.today().strftime("%Y%m%d")
query_date: str = pyautogui.prompt(title='조회할 날짜', text=f'조회할 날짜를 입력하세요. 예 : {today}', default=today)

##### 보고일자 계산

In [10]:
# 작성일자(조회일자)가 금요일이면 다음주 월요일이 보고일자
if datetime.strptime(query_date, '%Y%m%d').weekday() == 4:
    report_date: str = (datetime.strptime(query_date, '%Y%m%d') + timedelta(days=3)).strftime("%Y%m%d")
else:
    report_date: str = (datetime.strptime(query_date, '%Y%m%d') + timedelta(days=1)).strftime("%Y%m%d")
    
report_date

'20220809'

In [11]:
while True:
    i: int = 0
    if len(query_date) != 8:
        i += 1
        pyautogui.alert(title='입력오류', text='날짜는 8자리 입니다.')
        query_date = pyautogui.prompt(title='조회할 날짜', text=f'조회할 날짜를 입력하세요. 예 : {today}', default=today)

    if i == 0:
        break 

##### 주간조회 시작일자 입력

In [12]:
# 지난주 토요일 구하기

diff: int = (date.today().weekday() - 5) % 7 # 0:월요일, 1:화요일, 2:수요일, 3:목요일, 4:금요일, 5:토요일, 6:일요일
last_sat: str = (date.today() - timedelta(days=diff)).strftime("%Y%m%d") # 대쉬를 뺀 형식으로 변경
print(last_sat)

20220806


In [13]:
week_start: str = pyautogui.prompt(title='주간조회 시작날짜', text=f'주간조회 시작날짜를 입력하세요. 예 : {last_sat}', default=last_sat)

In [14]:
while True:
    i: int = 0
    if len(week_start) != 8:
        i += 1
        pyautogui.alert(title='입력오류', text='날짜는 8자리 입니다.')
        week_start = pyautogui.prompt(title='주간조회 시작날짜', text=f'주간조회 시작날짜를 입력하세요. 예 : {today}', default=today)

    if i == 0:
        break 

##### 주간조회 종료일자 입력

In [15]:
week_end: str = pyautogui.prompt(title='주간조회 종료날짜', text=f'주간조회 종료날짜를 입력하세요. 예 : {today}', default=today)

In [16]:
while True:
    i: int = 0
    if len(week_end) != 8:
        i += 1
        pyautogui.alert(title='입력오류', text='날짜는 8자리 입니다.')
        week_end = pyautogui.prompt(title='주간조회 종료날짜', text=f'주간조회 종료날짜를 입력하세요. 예 : {today}', default=today)

    if i == 0:
        break 

In [17]:
confirm: str = pyautogui.confirm(title='알림', text='작업을 시작합니다.\n취소하면 작업을 중단합니다.')

if confirm == 'Cancel':
    sys.exit()

### 수행시간측정 시작

In [18]:
start_lab: float = timer()

In [19]:
# 나중에 사용할 수도.

etc_cust: str = '' # 일반업체
g2b_gb: str = '' # 입찰구분


"""
< 일반낙찰업체 리스트 (etc_cust 에 기입) >

A2  미치코런던
A3	세이트스코트
A4	에이스베이직
A5	프리모
A6	스쿨뱅크
A7	하이틴
A8	런던베이직
A9	이엠씨
B1	김설영
B2	아이니
B4	쎈텐
Z	기타


< 입찰구분 (g2b_gb 에 기입) >
G G2B
1 수의계약
2 협의구매
"""

if etc_cust == '':
    etc: str = ''
else:
    etc: str = f" and a.g2b_co_gb  in ('E','I','L','S','{etc_cust}') "

if g2b_gb == '':
    g2b: str = ''
else:
    g2b: str = f" and a.g2b_gb  = '{g2b_gb}' "

### SQL문

In [20]:
# with open("파일이름.sql", "rt", encoding="utf8") as f:
#     oracleSql = f.read()

# 길이가 길 경우 파일로 넘겨준다.

# oracleSql = '''
# select agen_code,
#        rawtohex(utl_raw.cast_to_raw(agen_name)) agen_name
# from i_agen_t
# where 1=1
# '''

#### 한글변환 함수
    - SQL문의 한글 컬럼에는 다음과 같이 함수로 처리
    - rawtohex(utl_raw.cast_to_raw(변환할 컬럼))

In [21]:
# US7ASCII -> CP949(완성형한글) 로 변환
def us7ascii_to_cp949(df: pd.DataFrame) -> pd.DataFrame:
    for index, byte_data in enumerate(df):
        if byte_data == None: # null 값이면 패스. 안하면 변환 에러난다.
            continue
        byte_data = binascii.unhexlify(df[index])  # 16진수 문자열 hexstr로 표현된 바이너리 데이터를 반환. 역함수는 b2a_hex()
        df[index] = byte_data.decode("cp949")  # 바이트 변환값 -> cp949(완성형 한글) 로 변환
    return df

#### 전자입찰현황 (특약별) 

In [22]:
def bid_tkyk(query_dt: str, season1: str, season2: str, etc_sql: str, g2b_sql: str) -> pd.DataFrame:
    j_query_dt: str = str(int(query_dt[:4])-1) + query_dt[4:] # 1년전
    j_season1: str = str(int(season1[0:2])-1) + season1[-1] # 지난시즌
    if len(season2) == 3:
        j_season2: str = str(int(season2[0:2])-1) + season2[-1]
    else:
        j_season2: str = ''
    
    oracleSql: str = f'''
SELECT   z.tkyk,
         Rawtohex(utl_raw.Cast_to_raw(tkyk_name)) tkyk_name,
         sort,
         SUM(z.is_cnt)        is_cnt,
         SUM(z.is_cnt2)       is_cnt2,
         SUM(z.i_cnt)         i_cnt,
         SUM(z.s_cnt)         s_cnt,
         SUM(z.e_cnt)         e_cnt,
         SUM(z.l_cnt)         l_cnt,
         SUM(z.etc_cnt)       etc_cnt,
         SUM(z.i_qty)         i_qty,
         SUM(z.s_qty)         s_qty,
         SUM(z.e_qty)         e_qty,
         SUM(z.l_qty)         l_qty,
         SUM(z.etc_qty)       etc_qty,
         SUM(z.f_tot_cnt)     f_tot_cnt,
         SUM(z.f_sch_cnt)     f_sch_cnt,
         SUM(z.f_end_amt)     f_end_amt,
         SUM(z.f_stand_amt)   f_stand_amt,
         SUM(z.f_i_sch_cnt)   f_i_sch_cnt,
         SUM(z.s_tot_cnt)     s_tot_cnt,
         SUM(z.s_sch_cnt)     s_sch_cnt,
         SUM(z.s_end_amt)     s_end_amt,
         SUM(z.s_stand_amt)   s_stand_amt,
         SUM(z.s_i_sch_cnt)   s_i_sch_cnt,
         SUM(z.j_is_cnt)      j_is_cnt,
         SUM(z.j_is_cnt2)     j_is_cnt2,
         SUM(z.j_i_cnt)       j_i_cnt,
         SUM(z.j_s_cnt)       j_s_cnt,
         SUM(z.j_e_cnt)       j_e_cnt,
         SUM(z.j_l_cnt)       j_l_cnt,
         SUM(z.j_etc_cnt)     j_etc_cnt,
         SUM(z.j_i_qty)       j_i_qty,
         SUM(z.j_s_qty)       j_s_qty,
         SUM(z.j_e_qty)       j_e_qty,
         SUM(z.j_l_qty)       j_l_qty,
         SUM(z.j_etc_qty)     j_etc_qty,
         SUM(z.j_f_tot_cnt)   j_f_tot_cnt,
         SUM(z.j_f_sch_cnt)   j_f_sch_cnt,
         SUM(z.j_f_end_amt)   j_f_end_amt,
         SUM(z.j_f_stand_amt) j_f_stand_amt,
         SUM(z.j_f_i_sch_cnt) j_f_i_sch_cnt,
         SUM(z.j_s_tot_cnt)   j_s_tot_cnt,
         SUM(z.j_s_sch_cnt)   j_s_sch_cnt,
         SUM(z.j_s_end_amt)   j_s_end_amt,
         SUM(z.j_s_stand_amt) j_s_stand_amt,
         SUM(z.j_s_i_sch_cnt) j_s_i_sch_cnt
FROM     (
                SELECT a.g2b_tkyk                      tkyk,
                       To_char(a.g2b_date, 'yyyymmdd') g2b_date,
                       0                               is_cnt,
                       Decode(a.g2b_sch_gb,
                              'C', 2,
                              1) is_cnt2,
                       Decode(a.g2b_co_gb,
                              'I', 1,
                              0) i_cnt,
                       Decode(a.g2b_co_gb,
                              'E', 1,
                              0) e_cnt,
                       Decode(a.g2b_co_gb,
                              'S', 1,
                              0) s_cnt,
                       Decode(a.g2b_co_gb,
                              'L', 1,
                              0) l_cnt,
                       Decode(a.g2b_co_gb,
                              'I', 0,
                              'E', 0,
                              'S', 0,
                              'L', 0,
                              1) etc_cnt,
                       Decode(a.g2b_co_gb,
                              'I', g2b_qty,
                              0) i_qty,
                       Decode(a.g2b_co_gb,
                              'E', g2b_qty,
                              0) e_qty,
                       Decode(a.g2b_co_gb,
                              'S', g2b_qty,
                              0) s_qty,
                       Decode(a.g2b_co_gb,
                              'L', g2b_qty,
                              0) l_qty,
                       Decode(a.g2b_co_gb,
                              'I', 0,
                              'E', 0,
                              'S', 0,
                              'L', 0,
                              g2b_qty) etc_qty,
                       0               f_tot_cnt,
                       0               f_sch_cnt,
                       0               f_end_amt,
                       0               f_stand_amt,
                       0               f_i_sch_cnt,
                       0               s_tot_cnt,
                       0               s_sch_cnt,
                       0               s_end_amt,
                       0               s_stand_amt,
                       0               s_i_sch_cnt,
                       0               j_is_cnt,
                       0               j_is_cnt2,
                       0               j_i_cnt,
                       0               j_e_cnt,
                       0               j_s_cnt,
                       0               j_l_cnt,
                       0               j_etc_cnt,
                       0               j_i_qty,
                       0               j_e_qty,
                       0               j_s_qty,
                       0               j_l_qty,
                       0               j_etc_qty,
                       0               j_f_tot_cnt,
                       0               j_f_sch_cnt,
                       0               j_f_end_amt,
                       0               j_f_stand_amt,
                       0               j_f_i_sch_cnt,
                       0               j_s_tot_cnt,
                       0               j_s_sch_cnt,
                       0               j_s_end_amt,
                       0               j_s_stand_amt,
                       0               j_s_i_sch_cnt
                FROM   i_sale_g2b_t a,
                       i_sch_com_t
                WHERE  schc_code (+) = a.g2b_school
                AND    a.g2b_end_gb = '9'
                AND    a.g2b_date <= To_date('{query_dt}', 'yyyymmdd')
                AND    (
                              a.g2b_quota1 IN ( '{season1}',
                                               '{season2}' )
                       OR     a.g2b_quota2 IN ( '{season1}',
                                               '{season2}' ) ) {etc_sql} {g2b_sql}
                UNION ALL
                SELECT a.g2b_tkyk                      tkyk,
                       to_char(a.g2b_date, 'yyyymmdd') g2b_date,
                       0                               is_cnt,
                       0                               is_cnt2,
                       0                               i_cnt,
                       0                               e_cnt,
                       0                               s_cnt,
                       0                               l_cnt,
                       0                               etc_cnt,
                       0                               i_qty,
                       0                               e_qty,
                       0                               s_qty,
                       0                               l_qty,
                       0                               etc_qty,
                       1                               f_tot_cnt,
                       a.g2b_qty                       f_sch_cnt,
                       a.g2b_end_amt                   f_end_amt,
                       a.g2b_stand_amt                 f_stand_amt,
                       decode(a.g2b_co_gb,
                              'I', a.g2b_qty,
                              0) f_i_sch_cnt,
                       0         s_tot_cnt,
                       0         s_sch_cnt,
                       0         s_end_amt,
                       0         s_stand_amt,
                       0         s_i_sch_cnt,
                       0         j_is_cnt,
                       0         j_is_cnt2,
                       0         j_i_cnt,
                       0         j_e_cnt,
                       0         j_s_cnt,
                       0         j_l_cnt,
                       0         j_etc_cnt,
                       0         j_i_qty,
                       0         j_e_qty,
                       0         j_s_qty,
                       0         j_l_qty,
                       0         j_etc_qty,
                       0         j_f_tot_cnt,
                       0         j_f_sch_cnt,
                       0         j_f_end_amt,
                       0         j_f_stand_amt,
                       0         j_f_i_sch_cnt,
                       0         j_s_tot_cnt,
                       0         j_s_sch_cnt,
                       0         j_s_end_amt,
                       0         j_s_stand_amt,
                       0         j_s_i_sch_cnt
                FROM   i_sale_g2b_t a,
                       i_sch_com_t
                WHERE  schc_code (+) = a.g2b_school
                AND    a.g2b_season IN ( 'N',
                                        'F',
                                        'A',
                                        'B',
                                        'C' )
                AND    a.g2b_end_gb = '9'
                AND    a.g2b_date <= to_date('{query_dt}', 'yyyymmdd')
                AND    (
                              a.g2b_quota1 IN ( '{season1}',
                                               '{season2}' )
                       OR     a.g2b_quota2 IN ( '{season1}',
                                               '{season2}' ) ) {etc_sql} {g2b_sql}
                UNION ALL
                SELECT a.g2b_tkyk,
                       to_char(a.g2b_date, 'yyyymmdd') g2b_date,
                       0                               is_cnt,
                       0                               is_cnt2,
                       0                               i_cnt,
                       0                               e_cnt,
                       0                               s_cnt,
                       0                               l_cnt,
                       0                               etc_cnt,
                       0                               i_qty,
                       0                               e_qty,
                       0                               s_qty,
                       0                               l_qty,
                       0                               etc_qty,
                       0                               f_tot_cnt,
                       0                               f_sch_cnt,
                       0                               f_end_amt,
                       0                               f_stand_amt,
                       0                               f_i_sch_cnt,
                       1                               s_tot_cnt,
                       a.g2b_qty                       s_sch_cnt,
                       a.g2b_end_amt                   s_end_amt,
                       a.g2b_stand_amt                 s_stand_amt,
                       decode(a.g2b_co_gb,
                              'I', a.g2b_qty,
                              0) s_i_sch_cnt,
                       0         j_is_cnt,
                       0         j_is_cnt2,
                       0         j_i_cnt,
                       0         j_e_cnt,
                       0         j_s_cnt,
                       0         j_l_cnt,
                       0         j_etc_cnt,
                       0         j_i_qty,
                       0         j_e_qty,
                       0         j_s_qty,
                       0         j_l_qty,
                       0         j_etc_qty,
                       0         j_f_tot_cnt,
                       0         j_f_sch_cnt,
                       0         j_f_end_amt,
                       0         j_f_stand_amt,
                       0         j_f_i_sch_cnt,
                       0         j_s_tot_cnt,
                       0         j_s_sch_cnt,
                       0         j_s_end_amt,
                       0         j_s_stand_amt,
                       0         j_s_i_sch_cnt
                FROM   i_sale_g2b_t a,
                       i_sch_com_t
                WHERE  schc_code (+) = a.g2b_school
                AND    a.g2b_season IN ( 'S',
                                        'A',
                                        'B',
                                        'C' )
                AND    a.g2b_end_gb = '9'
                AND    a.g2b_date <= to_date('{query_dt}', 'yyyymmdd')
                AND    (
                              a.g2b_quota1 IN ( '{season1}',
                                               '{season2}' )
                       OR     a.g2b_quota2 IN ( '{season1}',
                                               '{season2}' ) ) {etc_sql} {g2b_sql}
                UNION ALL
                SELECT a.g2b_tkyk                      tkyk,
                       to_char(a.g2b_date, 'yyyymmdd') g2b_date,
                       0                               is_cnt,
                       0                               is_cnt2,
                       0                               i_cnt,
                       0                               e_cnt,
                       0                               s_cnt,
                       0                               l_cnt,
                       0                               etc_cnt,
                       0                               i_qty,
                       0                               e_qty,
                       0                               s_qty,
                       0                               l_qty,
                       0                               etc_qty,
                       0                               f_tot_cnt,
                       0                               f_sch_cnt,
                       0                               f_end_amt,
                       0                               f_stand_amt,
                       0                               f_i_sch_cnt,
                       0                               s_tot_cnt,
                       0                               s_sch_cnt,
                       0                               s_end_amt,
                       0                               s_stand_amt,
                       0                               s_i_sch_cnt,
                       0                               j_is_cnt,
                       decode(a.g2b_sch_gb,
                              'C', 2,
                              1) j_is_cnt2,
                       decode(a.g2b_co_gb,
                              'I', 1,
                              0) j_i_cnt,
                       decode(a.g2b_co_gb,
                              'E', 1,
                              0) j_e_cnt,
                       decode(a.g2b_co_gb,
                              'S', 1,
                              0) j_s_cnt,
                       decode(a.g2b_co_gb,
                              'L', 1,
                              0) j_l_cnt,
                       decode(a.g2b_co_gb,
                              'I', 0,
                              'E', 0,
                              'S', 0,
                              'L', 0,
                              1) j_etc_cnt,
                       decode(a.g2b_co_gb,
                              'I', g2b_qty,
                              0) j_i_qty,
                       decode(a.g2b_co_gb,
                              'E', g2b_qty,
                              0) j_e_qty,
                       decode(a.g2b_co_gb,
                              'S', g2b_qty,
                              0) j_s_qty,
                       decode(a.g2b_co_gb,
                              'L', g2b_qty,
                              0) j_l_qty,
                       decode(a.g2b_co_gb,
                              'I', 0,
                              'E', 0,
                              'S', 0,
                              'L', 0,
                              g2b_qty) j_etc_qty,
                       0               j_f_tot_cnt,
                       0               j_f_sch_cnt,
                       0               j_f_end_amt,
                       0               j_f_stand_amt,
                       0               j_f_i_sch_cnt,
                       0               j_s_tot_cnt,
                       0               j_s_sch_cnt,
                       0               j_s_end_amt,
                       0               vs_stand_amt,
                       0               j_s_i_sch_cnt
                FROM   i_sale_g2b_t a,
                       i_sch_com_t
                WHERE  schc_code (+) = a.g2b_school
                AND    a.g2b_end_gb = '9'
                AND    a.g2b_date <= to_date('{j_query_dt}', 'yyyymmdd')
                AND    (
                              a.g2b_quota1 IN ( '{j_season1}',
                                               '{j_season2}' )
                       OR     a.g2b_quota2 IN ( '{j_season1}',
                                               '{j_season2}' ) ) {etc_sql} {g2b_sql}
                UNION ALL
                SELECT a.g2b_tkyk                      tkyk,
                       to_char(a.g2b_date, 'yyyymmdd') g2b_date,
                       0                               is_cnt,
                       0                               is_cnt2,
                       0                               i_cnt,
                       0                               e_cnt,
                       0                               s_cnt,
                       0                               l_cnt,
                       0                               etc_cnt,
                       0                               i_qty,
                       0                               e_qty,
                       0                               s_qty,
                       0                               l_qty,
                       0                               etc_qty,
                       0                               f_tot_cnt,
                       0                               f_sch_cnt,
                       0                               f_end_amt,
                       0                               f_stand_amt,
                       0                               f_i_sch_cnt,
                       0                               s_tot_cnt,
                       0                               s_sch_cnt,
                       0                               s_end_amt,
                       0                               s_stand_amt,
                       0                               s_i_sch_cnt,
                       0                               j_is_cnt,
                       0                               j_is_cnt2,
                       0                               j_i_cnt,
                       0                               j_e_cnt,
                       0                               j_s_cnt,
                       0                               j_l_cnt,
                       0                               j_etc_cnt,
                       0                               j_i_qty,
                       0                               j_e_qty,
                       0                               j_s_qty,
                       0                               j_l_qty,
                       0                               j_etc_qty,
                       1                               j_f_tot_cnt,
                       a.g2b_qty                       j_f_sch_cnt,
                       a.g2b_end_amt                   j_f_end_amt,
                       a.g2b_stand_amt                 j_f_stand_amt,
                       decode(a.g2b_co_gb,
                              'I', a.g2b_qty,
                              0) j_f_i_sch_cnt,
                       0         j_s_tot_cnt,
                       0         j_s_sch_cnt,
                       0         j_s_end_amt,
                       0         j_s_stand_amt,
                       0         j_s_i_sch_cnt
                FROM   i_sale_g2b_t a,
                       i_sch_com_t
                WHERE  schc_code (+) = a.g2b_school
                AND    a.g2b_season IN ( 'N',
                                        'F',
                                        'A',
                                        'B',
                                        'C' )
                AND    a.g2b_end_gb = '9'
                AND    a.g2b_date <= to_date('{j_query_dt}', 'yyyymmdd')
                AND    (
                              a.g2b_quota1 IN ( '{j_season1}',
                                               '{j_season2}' )
                       OR     a.g2b_quota2 IN ( '{j_season1}',
                                               '{j_season2}' ) ) {etc_sql} {g2b_sql}
                UNION ALL
                SELECT a.g2b_tkyk,
                       to_char(a.g2b_date, 'yyyymmdd') g2b_date,
                       0                               is_cnt,
                       0                               is_cnt2,
                       0                               i_cnt,
                       0                               e_cnt,
                       0                               s_cnt,
                       0                               l_cnt,
                       0                               etc_cnt,
                       0                               i_qty,
                       0                               e_qty,
                       0                               s_qty,
                       0                               l_qty,
                       0                               etc_qty,
                       0                               f_tot_cnt,
                       0                               f_sch_cnt,
                       0                               f_end_amt,
                       0                               f_stand_amt,
                       0                               f_i_sch_cnt,
                       0                               s_tot_cnt,
                       0                               s_sch_cnt,
                       0                               s_end_amt,
                       0                               s_stand_amt,
                       0                               s_i_sch_cnt,
                       0                               j_is_cnt,
                       0                               j_is_cnt2,
                       0                               j_i_cnt,
                       0                               j_e_cnt,
                       0                               j_s_cnt,
                       0                               j_l_cnt,
                       0                               j_etc_cnt,
                       0                               j_i_qty,
                       0                               j_e_qty,
                       0                               j_s_qty,
                       0                               j_l_qty,
                       0                               j_etc_qty,
                       0                               j_f_tot_cnt,
                       0                               j_f_sch_cnt,
                       0                               j_f_end_amt,
                       0                               j_f_stand_amt,
                       0                               j_f_i_sch_cnt,
                       1                               j_s_tot_cnt,
                       a.g2b_qty                       j_s_sch_cnt,
                       a.g2b_end_amt                   j_s_end_amt,
                       a.g2b_stand_amt                 j_s_stand_amt,
                       decode(a.g2b_co_gb,
                              'I', a.g2b_qty,
                              0) j_s_i_sch_cnt
                FROM   i_sale_g2b_t a,
                       i_sch_com_t
                WHERE  schc_code (+) = a.g2b_school
                AND    a.g2b_season IN ( 'S',
                                        'A',
                                        'B',
                                        'C' )
                AND    a.g2b_end_gb = '9'
                AND    a.g2b_date <= to_date('{j_query_dt}', 'yyyymmdd')
                AND    (
                              a.g2b_quota1 IN ( '{j_season1}',
                                               '{j_season2}' )
                       OR     a.g2b_quota2 IN ( '{j_season1}',
                                               '{j_season2}' ) ) {etc_sql} {g2b_sql} ) z,
         i_tkyk_t
WHERE    z.tkyk = tkyk_code
GROUP BY z.tkyk,
         tkyk_name,
         SORT
ORDER BY SORT
'''
    df: pd.DataFrame = pd.read_sql_query(oracleSql, engine)
    df_temp: pd.DataFrame = df['tkyk_name'].copy()
    df['tkyk_name'] = us7ascii_to_cp949(df_temp)
    
    df['quota'] = f'{season1}{season2}'
    df = df[['quota', 'tkyk', 'sort', 'tkyk_name', 'i_cnt', 's_cnt', 'e_cnt', 'l_cnt', 'etc_cnt', 'i_qty', 's_qty', 'e_qty', 'l_qty', 'etc_qty']]
    return df

##### N + F 시즌

In [23]:
df_bid_tkyk_NF: pd.DataFrame = bid_tkyk(query_date, N_season, F_season, etc, g2b)
df_bid_tkyk_NF

,quota,tkyk,sort,tkyk_name,i_cnt,s_cnt,e_cnt,l_cnt,etc_cnt,i_qty,s_qty,e_qty,l_qty,etc_qty
0,23N22F,C,1,서울상권,5,7,4,6,10,885,1248,798,641,1922
1,23N22F,H,2,중부상권,47,52,29,37,52,7255,8908,5041,6539,9595
2,23N22F,D,3,대전상권,10,10,6,4,10,987,1609,1242,555,962
3,23N22F,L,4,광주상권,50,50,39,49,60,7358,6812,5443,6812,8455
4,23N22F,I,5,대구상권,41,34,36,26,43,6426,5066,5709,4319,7857
5,23N22F,R,6,부산상권,23,7,10,9,20,2177,707,1225,1313,3603


##### 없는 상권 추가하기 (함수)

In [24]:
tkyk_name: List[str] = ['서울상권', '중부상권', '대전상권', '광주상권', '대구상권', '부산상권']
tkyk_code: List[str] = ['C', 'H', 'D', 'L', 'I', 'R']
tkyk_sort: List[int] = [1, 2, 3, 4, 5, 6]

def add_tkyk(df: pd.DataFrame) -> pd.DataFrame:
    tkyk_lost: list = list(set(tkyk_sort) - set(df['sort'].tolist()) )
    for t in tkyk_lost:
        tkyk_insert: list = []
        tkyk_insert.append(df['quota'][0])
        tkyk_insert.append(tkyk_code[t-1])
        tkyk_insert.append(tkyk_sort[t-1])
        tkyk_insert.append(tkyk_name[t-1])
        tkyk_insert.append(0)
        tkyk_insert.append(0)
        tkyk_insert.append(0)
        tkyk_insert.append(0)
        tkyk_insert.append(0)
        tkyk_insert.append(0)
        tkyk_insert.append(0)
        tkyk_insert.append(0)
        tkyk_insert.append(0)
        tkyk_insert.append(0)
        df.loc[len(df)] = tkyk_insert
    df = df.sort_values('sort').reset_index(drop=True)
    return df

In [25]:
df_bid_tkyk_NF: pd.DataFrame = add_tkyk(df_bid_tkyk_NF)
df_bid_tkyk_NF

,quota,tkyk,sort,tkyk_name,i_cnt,s_cnt,e_cnt,l_cnt,etc_cnt,i_qty,s_qty,e_qty,l_qty,etc_qty
0,23N22F,C,1,서울상권,5,7,4,6,10,885,1248,798,641,1922
1,23N22F,H,2,중부상권,47,52,29,37,52,7255,8908,5041,6539,9595
2,23N22F,D,3,대전상권,10,10,6,4,10,987,1609,1242,555,962
3,23N22F,L,4,광주상권,50,50,39,49,60,7358,6812,5443,6812,8455
4,23N22F,I,5,대구상권,41,34,36,26,43,6426,5066,5709,4319,7857
5,23N22F,R,6,부산상권,23,7,10,9,20,2177,707,1225,1313,3603


##### 붙여넣을 합계 데이터

In [26]:
df_bid_tkyk_NF_ISEL_sum: pd.DataFrame = df_bid_tkyk_NF[['i_cnt', 'i_qty', 's_cnt', 's_qty', 'e_cnt', 'e_qty', 'l_cnt', 'l_qty']].copy()
df_bid_tkyk_NF_ISEL_sum

,i_cnt,i_qty,s_cnt,s_qty,e_cnt,e_qty,l_cnt,l_qty
0,5,885,7,1248,4,798,6,641
1,47,7255,52,8908,29,5041,37,6539
2,10,987,10,1609,6,1242,4,555
3,50,7358,50,6812,39,5443,49,6812
4,41,6426,34,5066,36,5709,26,4319
5,23,2177,7,707,10,1225,9,1313


In [27]:
df_bid_tkyk_NF_ISEL_sum['sum_cnt'] = (df_bid_tkyk_NF_ISEL_sum['i_cnt'] +
                                      df_bid_tkyk_NF_ISEL_sum['s_cnt'] +
                                      df_bid_tkyk_NF_ISEL_sum['e_cnt'] +
                                      df_bid_tkyk_NF_ISEL_sum['l_cnt'])

df_bid_tkyk_NF_ISEL_sum['sum_qty'] = (df_bid_tkyk_NF_ISEL_sum['i_qty'] +
                                      df_bid_tkyk_NF_ISEL_sum['s_qty'] +
                                      df_bid_tkyk_NF_ISEL_sum['e_qty'] +
                                      df_bid_tkyk_NF_ISEL_sum['l_qty'])
df_bid_tkyk_NF_ISEL_sum

,i_cnt,i_qty,s_cnt,s_qty,e_cnt,e_qty,l_cnt,l_qty,sum_cnt,sum_qty
0,5,885,7,1248,4,798,6,641,22,3572
1,47,7255,52,8908,29,5041,37,6539,165,27743
2,10,987,10,1609,6,1242,4,555,30,4393
3,50,7358,50,6812,39,5443,49,6812,188,26425
4,41,6426,34,5066,36,5709,26,4319,137,21520
5,23,2177,7,707,10,1225,9,1313,49,5422


In [28]:
df_bid_tkyk_NF_ISEL_sum.loc[len(df_bid_tkyk_NF_ISEL_sum)] = df_bid_tkyk_NF_ISEL_sum.sum()
df_bid_tkyk_NF_ISEL_sum

,i_cnt,i_qty,s_cnt,s_qty,e_cnt,e_qty,l_cnt,l_qty,sum_cnt,sum_qty
0,5,885,7,1248,4,798,6,641,22,3572
1,47,7255,52,8908,29,5041,37,6539,165,27743
2,10,987,10,1609,6,1242,4,555,30,4393
3,50,7358,50,6812,39,5443,49,6812,188,26425
4,41,6426,34,5066,36,5709,26,4319,137,21520
5,23,2177,7,707,10,1225,9,1313,49,5422
6,176,25088,160,24350,124,19458,131,20179,591,89075


##### F 시즌

In [29]:
df_bid_tkyk_F: pd.DataFrame = bid_tkyk(query_date, F_season, '', etc, g2b)
df_bid_tkyk_F

,quota,tkyk,sort,tkyk_name,i_cnt,s_cnt,e_cnt,l_cnt,etc_cnt,i_qty,s_qty,e_qty,l_qty,etc_qty
0,22F,C,1,서울상권,1,1,0,0,1,140,246,0,0,323
1,22F,H,2,중부상권,37,37,23,25,28,4762,5387,3353,4246,3790
2,22F,D,3,대전상권,0,0,0,0,0,0,0,0,0,0
3,22F,L,4,광주상권,48,50,38,48,59,6887,6812,5243,6556,8065
4,22F,I,5,대구상권,41,34,36,26,43,6426,5066,5709,4319,7857
5,22F,R,6,부산상권,23,7,10,8,16,2177,707,1225,1169,2818


In [30]:
df_bid_tkyk_F: pd.DataFrame = add_tkyk(df_bid_tkyk_F)
df_bid_tkyk_F

,quota,tkyk,sort,tkyk_name,i_cnt,s_cnt,e_cnt,l_cnt,etc_cnt,i_qty,s_qty,e_qty,l_qty,etc_qty
0,22F,C,1,서울상권,1,1,0,0,1,140,246,0,0,323
1,22F,H,2,중부상권,37,37,23,25,28,4762,5387,3353,4246,3790
2,22F,D,3,대전상권,0,0,0,0,0,0,0,0,0,0
3,22F,L,4,광주상권,48,50,38,48,59,6887,6812,5243,6556,8065
4,22F,I,5,대구상권,41,34,36,26,43,6426,5066,5709,4319,7857
5,22F,R,6,부산상권,23,7,10,8,16,2177,707,1225,1169,2818


##### 붙여넣을 합계 데이터

In [31]:
df_bid_tkyk_F_ISEL_sum: pd.DataFrame = df_bid_tkyk_F[['i_cnt', 'i_qty', 's_cnt', 's_qty', 'e_cnt', 'e_qty', 'l_cnt', 'l_qty']].copy()
df_bid_tkyk_F_ISEL_sum

,i_cnt,i_qty,s_cnt,s_qty,e_cnt,e_qty,l_cnt,l_qty
0,1,140,1,246,0,0,0,0
1,37,4762,37,5387,23,3353,25,4246
2,0,0,0,0,0,0,0,0
3,48,6887,50,6812,38,5243,48,6556
4,41,6426,34,5066,36,5709,26,4319
5,23,2177,7,707,10,1225,8,1169


In [32]:
df_bid_tkyk_F_ISEL_sum.loc[len(df_bid_tkyk_F_ISEL_sum)] = df_bid_tkyk_F_ISEL_sum.sum()
df_bid_tkyk_F_ISEL_sum

,i_cnt,i_qty,s_cnt,s_qty,e_cnt,e_qty,l_cnt,l_qty
0,1,140,1,246,0,0,0,0
1,37,4762,37,5387,23,3353,25,4246
2,0,0,0,0,0,0,0,0
3,48,6887,50,6812,38,5243,48,6556
4,41,6426,34,5066,36,5709,26,4319
5,23,2177,7,707,10,1225,8,1169
6,150,20392,129,18218,107,15530,107,16290


##### S 시즌

In [33]:
df_bid_tkyk_S: pd.DataFrame = bid_tkyk(query_date, S_season, '', etc, g2b)
df_bid_tkyk_S

,quota,tkyk,sort,tkyk_name,i_cnt,s_cnt,e_cnt,l_cnt,etc_cnt,i_qty,s_qty,e_qty,l_qty,etc_qty
0,23S,C,1,서울상권,4,6,4,6,9,745,1002,798,641,1599
1,23S,H,2,중부상권,10,15,6,12,24,2493,3521,1688,2293,5805
2,23S,D,3,대전상권,10,10,6,4,11,987,1609,1242,555,966
3,23S,I,5,대구상권,0,0,0,0,0,0,0,0,0,0
4,23S,R,6,부산상권,0,0,0,1,4,0,0,0,144,785


In [34]:
df_bid_tkyk_S: pd.DataFrame = add_tkyk(df_bid_tkyk_S)
df_bid_tkyk_S

,quota,tkyk,sort,tkyk_name,i_cnt,s_cnt,e_cnt,l_cnt,etc_cnt,i_qty,s_qty,e_qty,l_qty,etc_qty
0,23S,C,1,서울상권,4,6,4,6,9,745,1002,798,641,1599
1,23S,H,2,중부상권,10,15,6,12,24,2493,3521,1688,2293,5805
2,23S,D,3,대전상권,10,10,6,4,11,987,1609,1242,555,966
3,23S,L,4,광주상권,0,0,0,0,0,0,0,0,0,0
4,23S,I,5,대구상권,0,0,0,0,0,0,0,0,0,0
5,23S,R,6,부산상권,0,0,0,1,4,0,0,0,144,785


##### 붙여넣을 합계 데이터

In [35]:
df_bid_tkyk_S_ISEL_sum: pd.DataFrame = df_bid_tkyk_S[['i_cnt', 'i_qty', 's_cnt', 's_qty', 'e_cnt', 'e_qty', 'l_cnt', 'l_qty']].copy()
df_bid_tkyk_S_ISEL_sum

,i_cnt,i_qty,s_cnt,s_qty,e_cnt,e_qty,l_cnt,l_qty
0,4,745,6,1002,4,798,6,641
1,10,2493,15,3521,6,1688,12,2293
2,10,987,10,1609,6,1242,4,555
3,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,1,144


In [36]:
df_bid_tkyk_S_ISEL_sum['sum_cnt'] = (df_bid_tkyk_S_ISEL_sum['i_cnt'] +
                                     df_bid_tkyk_S_ISEL_sum['s_cnt'] +
                                     df_bid_tkyk_S_ISEL_sum['e_cnt'] +
                                     df_bid_tkyk_S_ISEL_sum['l_cnt'])

df_bid_tkyk_S_ISEL_sum['sum_qty'] = (df_bid_tkyk_S_ISEL_sum['i_qty'] +
                                     df_bid_tkyk_S_ISEL_sum['s_qty'] +
                                     df_bid_tkyk_S_ISEL_sum['e_qty'] +
                                     df_bid_tkyk_S_ISEL_sum['l_qty'])
df_bid_tkyk_S_ISEL_sum

,i_cnt,i_qty,s_cnt,s_qty,e_cnt,e_qty,l_cnt,l_qty,sum_cnt,sum_qty
0,4,745,6,1002,4,798,6,641,20,3186
1,10,2493,15,3521,6,1688,12,2293,43,9995
2,10,987,10,1609,6,1242,4,555,30,4393
3,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,1,144,1,144


In [37]:
df_bid_tkyk_S_ISEL_sum.loc[len(df_bid_tkyk_S_ISEL_sum)] = df_bid_tkyk_S_ISEL_sum.sum()
df_bid_tkyk_S_ISEL_sum

,i_cnt,i_qty,s_cnt,s_qty,e_cnt,e_qty,l_cnt,l_qty,sum_cnt,sum_qty
0,4,745,6,1002,4,798,6,641,20,3186
1,10,2493,15,3521,6,1688,12,2293,43,9995
2,10,987,10,1609,6,1242,4,555,30,4393
3,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,1,144,1,144
6,24,4225,31,6132,16,3728,23,3633,94,17718


##### 붙여넣을 합계 테이터 (M.S)

In [38]:
df_bid_tkyk_NF

,quota,tkyk,sort,tkyk_name,i_cnt,s_cnt,e_cnt,l_cnt,etc_cnt,i_qty,s_qty,e_qty,l_qty,etc_qty
0,23N22F,C,1,서울상권,5,7,4,6,10,885,1248,798,641,1922
1,23N22F,H,2,중부상권,47,52,29,37,52,7255,8908,5041,6539,9595
2,23N22F,D,3,대전상권,10,10,6,4,10,987,1609,1242,555,962
3,23N22F,L,4,광주상권,50,50,39,49,60,7358,6812,5443,6812,8455
4,23N22F,I,5,대구상권,41,34,36,26,43,6426,5066,5709,4319,7857
5,23N22F,R,6,부산상권,23,7,10,9,20,2177,707,1225,1313,3603


In [39]:
df_ms_total: pd.DataFrame = pd.DataFrame()
df_ms_total = pd.concat([df_ms_total,
                         df_bid_tkyk_NF.groupby('quota')[['i_cnt', 'i_qty', 's_cnt', 's_qty', 'e_cnt', 'e_qty', 'l_cnt', 'l_qty', 'etc_cnt', 'etc_qty']].agg(sum).T,
                         df_bid_tkyk_F.groupby('quota')[['i_cnt', 'i_qty', 's_cnt', 's_qty', 'e_cnt', 'e_qty', 'l_cnt', 'l_qty', 'etc_cnt', 'etc_qty']].agg(sum).T,
                         df_bid_tkyk_S.groupby('quota')[['i_cnt', 'i_qty', 's_cnt', 's_qty', 'e_cnt', 'e_qty', 'l_cnt', 'l_qty', 'etc_cnt', 'etc_qty']].agg(sum).T
                        ], axis=1)
df_ms_total[N_season] = df_ms_total[N_season + F_season] - df_ms_total[F_season]

# 집계기준용 컬럼
df_ms_total = df_ms_total.reset_index()
df_ms_total[['gbn1', 'gbn2']] = df_ms_total['index'].str.split('_', n=1, expand=True)

# 합계라인 추가
df_ms_total = pd.concat([df_ms_total, df_ms_total.groupby('gbn2')[[N_season + F_season, F_season, S_season, N_season]].agg(sum)])

df_ms_total

quota,index,23N22F,22F,23S,23N,gbn1,gbn2
0,i_cnt,176,150,24,26,i,cnt
1,i_qty,25088,20392,4225,4696,i,qty
2,s_cnt,160,129,31,31,s,cnt
3,s_qty,24350,18218,6132,6132,s,qty
4,e_cnt,124,107,16,17,e,cnt
5,e_qty,19458,15530,3728,3928,e,qty
6,l_cnt,131,107,23,24,l,cnt
7,l_qty,20179,16290,3633,3889,l,qty
8,etc_cnt,195,147,48,48,etc,cnt
9,etc_qty,32394,22853,9155,9541,etc,qty


In [40]:
i_lst: List[Union[str, float, int]] = ['',
                                       round(df_ms_total.loc[df_ms_total[df_ms_total['index'] == 'i_qty'].index[0], N_season + F_season] / df_ms_total.loc['qty', N_season + F_season], 3),
                                       round(df_ms_total.loc[df_ms_total[df_ms_total['index'] == 'i_qty'].index[0], F_season] / df_ms_total.loc['qty', F_season], 3),
                                       round(df_ms_total.loc[df_ms_total[df_ms_total['index'] == 'i_qty'].index[0], S_season] / df_ms_total.loc['qty', S_season], 3),
                                       round(df_ms_total.loc[df_ms_total[df_ms_total['index'] == 'i_qty'].index[0], N_season] / df_ms_total.loc['qty', N_season], 3),
                                       'i',
                                       ''
                                      ]

s_lst: List[Union[str, float, int]] = ['',
                                       round(df_ms_total.loc[df_ms_total[df_ms_total['index'] == 's_qty'].index[0], N_season + F_season] / df_ms_total.loc['qty', N_season + F_season], 3),
                                       round(df_ms_total.loc[df_ms_total[df_ms_total['index'] == 's_qty'].index[0], F_season] / df_ms_total.loc['qty', F_season], 3),
                                       round(df_ms_total.loc[df_ms_total[df_ms_total['index'] == 's_qty'].index[0], S_season] / df_ms_total.loc['qty', S_season], 3),
                                       round(df_ms_total.loc[df_ms_total[df_ms_total['index'] == 's_qty'].index[0], N_season] / df_ms_total.loc['qty', N_season], 3),
                                       's',
                                       ''
                                      ]

e_lst: List[Union[str, float, int]] = ['',
                                       round(df_ms_total.loc[df_ms_total[df_ms_total['index'] == 'e_qty'].index[0], N_season + F_season] / df_ms_total.loc['qty', N_season + F_season], 3),
                                       round(df_ms_total.loc[df_ms_total[df_ms_total['index'] == 'e_qty'].index[0], F_season] / df_ms_total.loc['qty', F_season], 3),
                                       round(df_ms_total.loc[df_ms_total[df_ms_total['index'] == 'e_qty'].index[0], S_season] / df_ms_total.loc['qty', S_season], 3),
                                       round(df_ms_total.loc[df_ms_total[df_ms_total['index'] == 'e_qty'].index[0], N_season] / df_ms_total.loc['qty', N_season], 3),
                                       'e',
                                       ''
                                      ]

l_lst: List[Union[str, float, int]] = ['',
                                       round(df_ms_total.loc[df_ms_total[df_ms_total['index'] == 'l_qty'].index[0], N_season + F_season] / df_ms_total.loc['qty', N_season + F_season], 3),
                                       round(df_ms_total.loc[df_ms_total[df_ms_total['index'] == 'l_qty'].index[0], F_season] / df_ms_total.loc['qty', F_season], 3),
                                       round(df_ms_total.loc[df_ms_total[df_ms_total['index'] == 'l_qty'].index[0], S_season] / df_ms_total.loc['qty', S_season], 3),
                                       round(df_ms_total.loc[df_ms_total[df_ms_total['index'] == 'l_qty'].index[0], N_season] / df_ms_total.loc['qty', N_season], 3),
                                       'l',
                                       ''
                                      ]

etc_lst: List[Union[str, float, int]] = ['',
                                         round(df_ms_total.loc[df_ms_total[df_ms_total['index'] == 'etc_qty'].index[0], N_season + F_season] / df_ms_total.loc['qty', N_season + F_season], 3),
                                         round(df_ms_total.loc[df_ms_total[df_ms_total['index'] == 'etc_qty'].index[0], F_season] / df_ms_total.loc['qty', F_season], 3),
                                         round(df_ms_total.loc[df_ms_total[df_ms_total['index'] == 'etc_qty'].index[0], S_season] / df_ms_total.loc['qty', S_season], 3),
                                         round(df_ms_total.loc[df_ms_total[df_ms_total['index'] == 'etc_qty'].index[0], N_season] / df_ms_total.loc['qty', N_season], 3),
                                         'etc',
                                         ''
                                        ]

i_lst

['', 0.207, 0.219, 0.157, 0.167, 'i', '']

In [41]:
df_ms_total = df_ms_total.reset_index(drop=True)

df_ms_total.loc[df_ms_total[df_ms_total['index'] == 'i_qty'].index[0]+0.5] = i_lst
df_ms_total.loc[df_ms_total[df_ms_total['index'] == 's_qty'].index[0]+0.5] = s_lst
df_ms_total.loc[df_ms_total[df_ms_total['index'] == 'e_qty'].index[0]+0.5] = e_lst
df_ms_total.loc[df_ms_total[df_ms_total['index'] == 'l_qty'].index[0]+0.5] = l_lst
df_ms_total.loc[df_ms_total[df_ms_total['index'] == 'etc_qty'].index[0]+0.5] = etc_lst

df_ms_total = df_ms_total.sort_index().reset_index(drop=True)

df_ms_total = df_ms_total[[N_season + F_season, N_season, S_season, F_season]]

df_ms_total

quota,23N22F,23N,23S,22F
0,176.000,26.000,24.000,150.000
1,25088.000,4696.000,4225.000,20392.000
2,0.207,0.167,0.157,0.219
3,160.000,31.000,31.000,129.000
4,24350.000,6132.000,6132.000,18218.000
5,0.200,0.218,0.228,0.195
6,124.000,17.000,16.000,107.000
7,19458.000,3928.000,3728.000,15530.000
8,0.160,0.139,0.139,0.166
9,131.000,24.000,23.000,107.000


#### 수주량 조회 (특약별) 

In [42]:
def suju_query(season1: str, season2: str, season3: str) -> pd.DataFrame:
    oracleSql: str = f'''
SELECT z.QUOTA,
       z.gubun,
       z.bokjong,
       z.tkyk,
       Rawtohex(utl_raw.Cast_to_raw(tkyk_name)) tkyk_name,
       sort,
       SUM(z.qty)                               qty
FROM   (SELECT '{season1}{season2}' QUOTA,
               '03'                 gubun,
               Max(master_bokjong)  bokjong,
               master_tkyk          tkyk,
               SUM(master_suju_qty) qty
        FROM   i_suju_master_t,
               i_sch_t
        WHERE  master_quota IN ( '{season1}', '{season2}' )
               AND master_jaepum = 'F'
               AND master_bokjong = 'H'
               AND master_tkyk IN ( 'C', 'D', 'H', 'I',
                                    'L', 'R' )
               AND master_status >= '03'
               AND master_status <= '60'
               AND sch_code(+) = master_school
               AND master_bokjong = sch_f_bokjong
        GROUP  BY master_tkyk
        UNION ALL
        SELECT '{season1}{season2}' QUOTA,
               '03'                 gubun,
               Max(master_bokjong)  bokjong,
               master_tkyk          tkyk,
               SUM(master_suju_qty) qty
        FROM   i_suju_master_t
        WHERE  master_quota IN ( '{season1}', '{season2}' )
               AND master_jaepum = 'H'
               AND master_bokjong = 'J'
               AND master_tkyk IN ( 'C', 'D', 'H', 'I',
                                    'L', 'R' )
               AND master_status >= '03'
               AND master_status <= '60'
        GROUP  BY master_tkyk
        UNION ALL
        SELECT '{season1}{season2}' QUOTA,
               '12'                 gubun,
               Max(master_bokjong)  bokjong,
               master_tkyk          tkyk,
               SUM(master_suju_qty) qty
        FROM   i_suju_master_t,
               i_sch_t
        WHERE  master_quota IN ( '{season1}', '{season2}' )
               AND master_jaepum = 'F'
               AND master_bokjong = 'H'
               AND master_tkyk IN ( 'C', 'D', 'H', 'I',
                                    'L', 'R' )
               AND master_status >= '12'
               AND master_status <= '60'
               AND sch_code(+) = master_school
               AND master_bokjong = sch_f_bokjong
        GROUP  BY master_tkyk
        UNION ALL
        SELECT '{season1}{season2}' QUOTA,
               '12'                 gubun,
               Max(master_bokjong)  bokjong,
               master_tkyk          tkyk,
               SUM(master_suju_qty) qty
        FROM   i_suju_master_t
        WHERE  master_quota IN ( '{season1}', '{season2}' )
               AND master_jaepum = 'H'
               AND master_bokjong = 'J'
               AND master_tkyk IN ( 'C', 'D', 'H', 'I',
                                    'L', 'R' )
               AND master_status >= '12'
               AND master_status <= '60'
        GROUP  BY master_tkyk
        UNION ALL
        SELECT '{season2}'          QUOTA,
               '03'                 gubun,
               Max(master_bokjong)  bokjong,
               master_tkyk          tkyk,
               SUM(master_suju_qty) qty
        FROM   i_suju_master_t,
               i_sch_t
        WHERE  master_quota IN ( '{season2}' )
               AND master_jaepum = 'F'
               AND master_bokjong = 'H'
               AND master_tkyk IN ( 'C', 'D', 'H', 'I',
                                    'L', 'R' )
               AND master_status >= '03'
               AND master_status <= '60'
               AND sch_code(+) = master_school
               AND master_bokjong = sch_f_bokjong
        GROUP  BY master_tkyk
        UNION ALL
        SELECT '{season2}'          QUOTA,
               '03'                 gubun,
               Max(master_bokjong)  bokjong,
               master_tkyk          tkyk,
               SUM(master_suju_qty) qty
        FROM   i_suju_master_t
        WHERE  master_quota IN ( '{season2}' )
               AND master_jaepum = 'H'
               AND master_bokjong = 'J'
               AND master_tkyk IN ( 'C', 'D', 'H', 'I',
                                    'L', 'R' )
               AND master_status >= '03'
               AND master_status <= '60'
        GROUP  BY master_tkyk
        UNION ALL
        SELECT '{season2}'          QUOTA,
               '12'                 gubun,
               Max(master_bokjong)  bokjong,
               master_tkyk          tkyk,
               SUM(master_suju_qty) qty
        FROM   i_suju_master_t,
               i_sch_t
        WHERE  master_quota IN ( '{season2}' )
               AND master_jaepum = 'F'
               AND master_bokjong = 'H'
               AND master_tkyk IN ( 'C', 'D', 'H', 'I',
                                    'L', 'R' )
               AND master_status >= '12'
               AND master_status <= '60'
               AND sch_code(+) = master_school
               AND master_bokjong = sch_f_bokjong
        GROUP  BY master_tkyk
        UNION ALL
        SELECT '{season2}'          QUOTA,
               '12'                 gubun,
               Max(master_bokjong)  bokjong,
               master_tkyk          tkyk,
               SUM(master_suju_qty) qty
        FROM   i_suju_master_t
        WHERE  master_quota IN ( '{season2}' )
               AND master_jaepum = 'H'
               AND master_bokjong = 'J'
               AND master_tkyk IN ( 'C', 'D', 'H', 'I',
                                    'L', 'R' )
               AND master_status >= '12'
               AND master_status <= '60'
        GROUP  BY master_tkyk
        UNION ALL
        SELECT '{season3}'          QUOTA,
               '03'                 gubun,
               'HA'                 bokjong,
               master_tkyk          tkyk,
               SUM(master_suju_qty) qty
        FROM   i_suju_master_t,
               i_cod_t tt
        WHERE  master_quota IN ( '{season3}' )
               AND master_jaepum = 'H'
               AND master_tkyk IN ( 'C', 'D', 'H', 'I',
                                    'L', 'R' )
               AND master_status >= '03'
               AND master_status <= '60'
               AND tt.cod_code = master_bokjong
               AND tt.cod_gbn_code = '01'
               AND Nvl(tt.cod_etc2, '') = 'H'
               AND Nvl(tt.cod_etc3, '') IN ( 'S', 'T' )
        GROUP  BY master_tkyk
        UNION ALL
        SELECT '{season3}'          QUOTA,
               '12'                 gubun,
               'HA'                 bokjong,
               master_tkyk          tkyk,
               SUM(master_suju_qty) qty
        FROM   i_suju_master_t,
               i_cod_t tt
        WHERE  master_quota IN ( '{season3}' )
               AND master_jaepum = 'H'
               AND master_tkyk IN ( 'C', 'D', 'H', 'I',
                                    'L', 'R' )
               AND master_status >= '12'
               AND master_status <= '60'
               AND tt.cod_code = master_bokjong
               AND tt.cod_gbn_code = '01'
               AND Nvl(tt.cod_etc2, '') = 'H'
               AND Nvl(tt.cod_etc3, '') IN ( 'S', 'T' )
        GROUP  BY master_tkyk) z,
       i_tkyk_t
WHERE  z.tkyk = tkyk_code
GROUP  BY z.QUOTA,
          z.gubun,
          z.bokjong,
          z.tkyk,
          tkyk_name,
          sort
ORDER  BY z.QUOTA,
          z.gubun,
          z.bokjong DESC,
          sort
'''
    df: pd.DataFrame = pd.read_sql_query(oracleSql, engine)
    df_temp: pd.DataFrame = df['tkyk_name'].copy()
    df['tkyk_name'] = us7ascii_to_cp949(df_temp)
    
    df['gubun'] = df['gubun'].str.replace('03', '수주량').str.replace('12', '해제량')
    return df

In [43]:
df_suju_qty:pd.DataFrame = suju_query(N_season, F_season, S_season)
df_suju_qty.head()

,quota,gubun,bokjong,tkyk,tkyk_name,sort,qty
0,22F,수주량,J,C,서울상권,1,177
1,22F,수주량,J,H,중부상권,2,4385
2,22F,수주량,J,D,대전상권,3,54
3,22F,수주량,J,L,광주상권,4,3528
4,22F,수주량,J,I,대구상권,5,4688


#### 수주량 전주간 조회 (수주일 기준 / 영업확정일 기준)

     - ST는 03 ~ 60 으로 고정
     - 동복은 복종 J 기준, 하복은 하의 기준

In [44]:
def suju_query_week(season1: str, season2: str, season3: str, start_dt: str, end_dt: str) -> pd.DataFrame:
    oracleSql = f'''
SELECT z.quota,
       z.gubun,
       z.bokjong,
       Sum(z.qty) qty
FROM (
      SELECT '{season1}{season2}' quota, '03' gubun, Max(master_bokjong) bokjong, Sum(master_suju_qty) qty
        FROM i_suju_master_t
       WHERE master_quota IN ('{season1}', '{season2}')
         AND master_jaepum = 'H'
         AND master_bokjong = 'J'
         AND master_tkyk IN ('C','D','H','I','L','R')
         AND master_status >= '03'
         AND master_status <= '60'
         AND master_suju_date >= to_date('{start_dt}','yyyy/mm/dd')
         AND master_suju_date <= to_date('{end_dt}','yyyy/mm/dd')
      UNION ALL
      SELECT '{season1}{season2}' quota, '12' gubun, Max(master_bokjong) bokjong, Sum(master_suju_qty) qty
        FROM i_suju_master_t
       WHERE master_quota IN ('{season1}', '{season2}')
         AND master_jaepum = 'H'
         AND master_bokjong = 'J'
         AND master_tkyk IN ('C','D','H','I','L','R')
         AND master_status >= '03'
         AND master_status <= '60'
         AND master_appv_end_dt >= to_date('{start_dt}','yyyy/mm/dd')
         AND master_appv_end_dt <= to_date('{end_dt}','yyyy/mm/dd')
      UNION ALL
      SELECT '{season2}' quota, '03' gubun, Max(master_bokjong) bokjong, Sum(master_suju_qty) qty
        FROM i_suju_master_t
       WHERE master_quota IN ('{season2}')
         AND master_jaepum = 'H'
         AND master_bokjong = 'J'
         AND master_tkyk IN ('C','D','H','I','L','R')
         AND master_status >= '03'
         AND master_status <= '60'
         AND master_suju_date >= to_date('{start_dt}','yyyy/mm/dd')
         AND master_suju_date <= to_date('{end_dt}','yyyy/mm/dd')
      UNION ALL
      SELECT '{season2}' quota, '12' gubun, Max(master_bokjong) bokjong, Sum(master_suju_qty) qty
        FROM i_suju_master_t
       WHERE master_quota IN ('{season2}')
         AND master_jaepum = 'H'
         AND master_bokjong = 'J'
         AND master_tkyk IN ('C','D','H','I','L','R')
         AND master_status >= '03'
         AND master_status <= '60'
         AND master_appv_end_dt >= to_date('{start_dt}','yyyy/mm/dd')
         AND master_appv_end_dt <= to_date('{end_dt}','yyyy/mm/dd')
      UNION ALL
      SELECT '{season3}' quota, '03' gubun, 'HA' bokjong, Sum(master_suju_qty) qty
        FROM i_suju_master_t, i_cod_t tt
       WHERE master_quota IN ('{season3}')
         AND master_jaepum = 'H'
         AND master_tkyk IN ('C','D','H','I','L','R')
         AND master_status >= '03'
         AND master_status <= '60'
         AND tt.cod_code = master_bokjong
         AND tt.cod_gbn_code  = '01'
         AND nvl(tt.cod_etc2,'') = 'H'
         AND nvl(tt.cod_etc3,'') IN ('S','T')
         AND master_suju_date >= to_date('{start_dt}','yyyy/mm/dd')
         AND master_suju_date <= to_date('{end_dt}','yyyy/mm/dd')
      UNION ALL
      SELECT '{season3}' quota, '12' gubun, 'HA' bokjong, Sum(master_suju_qty) qty
        FROM i_suju_master_t, i_cod_t tt
       WHERE master_quota IN ('{season3}')
         AND master_jaepum = 'H'
         AND master_tkyk IN ('C','D','H','I','L','R')
         AND master_status >= '03'
         AND master_status <= '60'
         AND tt.cod_code = master_bokjong
         AND tt.cod_gbn_code  = '01'
         AND nvl(tt.cod_etc2,'') = 'H'
         AND nvl(tt.cod_etc3,'') IN ('S','T')
         AND master_appv_end_dt >= to_date('{start_dt}','yyyy/mm/dd')
         AND master_appv_end_dt <= to_date('{end_dt}','yyyy/mm/dd')
         ) z
GROUP BY z.quota, z.gubun, z.bokjong
ORDER BY z.quota, z.gubun, z.bokjong desc
'''
    df: pd.DataFrame = pd.read_sql_query(oracleSql, engine)
    df['gubun'] = df['gubun'].str.replace('03', '수주량').str.replace('12', '해제량')
    return df

In [45]:
df_suju_qty_week: pd.DataFrame = suju_query_week(N_season, F_season, S_season, week_start, week_end)
df_suju_qty_week

,quota,gubun,bokjong,qty
0,22F,수주량,None,NaN
1,22F,해제량,J,102.0
2,23N22F,수주량,None,NaN
3,23N22F,해제량,J,220.0
4,23S,수주량,HA,NaN
5,23S,해제량,HA,NaN


## 엑셀파일 작업 시작

### 엑셀기본 라이브러리

In [46]:
import xlwings as xw

### 환경변수

In [47]:
# file_path1: str = "C:/Users/Administrator/PythonDataWorkspace/영업팀 일일자료/"
file_path2: str = "C:/Users/Administrator/Documents/"

report_date2: str = report_date[:4] + '-' + report_date[4:6] + '-' + report_date[6:]
report_date2, report_date

('2022-08-09', '20220809')

### '23년 학교주관구매 M.S 현황.xlsx 파일 오픈

#### 엑셀 인스턴스 생성

In [48]:
app: 'xlwings.main.App' = xw.App(visible=True) # 실행과정 보이게
# app: 'xlwings.main.App' = xw.App(visible=False) # 실행과정 안보이게

#### M.S 시트 지정

In [49]:
try:
    wb = xw.Book(f"./'23년 학교주관구매 M.S 현황.xlsx") # type: xlwings.main.Book
    sh_MS = wb.sheets['M.S'] # type: xlwings.main.Sheet
except FileNotFoundError:
    print("같은 폴더안에 \n'23년 학교주관구매 M.S 현황.xlsx \n파일이 없습니다.\n\n프로그램을 종료합니다.")
    pyautogui.alert(title='FileNotFoundError', text="같은 폴더안에 \n'23년 학교주관구매 M.S 현황.xlsx \n파일이 없습니다.\n\n프로그램을 종료합니다.")
    sys.exit()

#### 동복 시트 지정

In [50]:
sh_NF = wb.sheets['# 동복'] # type: xlwings.main.Sheet
sh_NF['J4:P20'].value = sh_NF['B4:H20'].value # 전일 데이터를 숨기기 된 컬럼에 값만 복사

#### 하복 시트 지정

In [51]:
sh_S = wb.sheets['# 하복'] # type: xlwings.main.Sheet
sh_S['J4:P20'].value = sh_S['B4:H20'].value # 전일 데이터를 숨기기 된 컬럼에 값만 복사

#### F 시즌 시트 지정

In [52]:
sh_F = wb.sheets[f'# {F_season}'] # type: xlwings.main.Sheet
sh_F['J4:P20'].value = sh_F['B4:H20'].value # 전일 데이터를 숨기기 된 컬럼에 값만 복사

#### N + F 시즌 -> M.S 시트에 기록

In [53]:
df_bid_tkyk_NF

,quota,tkyk,sort,tkyk_name,i_cnt,s_cnt,e_cnt,l_cnt,etc_cnt,i_qty,s_qty,e_qty,l_qty,etc_qty
0,23N22F,C,1,서울상권,5,7,4,6,10,885,1248,798,641,1922
1,23N22F,H,2,중부상권,47,52,29,37,52,7255,8908,5041,6539,9595
2,23N22F,D,3,대전상권,10,10,6,4,10,987,1609,1242,555,962
3,23N22F,L,4,광주상권,50,50,39,49,60,7358,6812,5443,6812,8455
4,23N22F,I,5,대구상권,41,34,36,26,43,6426,5066,5709,4319,7857
5,23N22F,R,6,부산상권,23,7,10,9,20,2177,707,1225,1313,3603


In [54]:
# len(df_bid_tkyk_NF[df_bid_tkyk_NF['tkyk_name'] == '서울상권'].index) == 1
df_bid_tkyk_NF[df_bid_tkyk_NF['tkyk_name'] == '광주상권'].index[0]

3

In [55]:
sh_MS['H2'].value = report_date2

# 서울상권
if len(df_bid_tkyk_NF[df_bid_tkyk_NF['tkyk_name'] == '서울상권'].index) == 1: # 서울상권 row가 1줄 존재하는가?
    index_tmp = df_bid_tkyk_NF[df_bid_tkyk_NF['tkyk_name'] == '서울상권'].index[0] # 서울상권 인덱스번호

    # 한 줄 추가할 때는 시리즈로 자동변환 되어서 강제로 데이터프레임 변경 후 축을 돌려서 입력함
    sh_MS['D4'].options(index=False, header=False).value = df_bid_tkyk_NF.loc[index_tmp, 'i_cnt':'etc_cnt'].to_frame().T
    sh_MS['D5'].options(index=False, header=False).value = df_bid_tkyk_NF.loc[index_tmp, 'i_qty':'etc_qty'].to_frame().T
else: # 조건을 만족하지 않을 때는 0으로 처리한다.
    sh_MS['D4'].value = [0, 0, 0, 0, 0]
    sh_MS['D5'].value = [0, 0, 0, 0, 0]


# 중부상권
if len(df_bid_tkyk_NF[df_bid_tkyk_NF['tkyk_name'] == '중부상권'].index) == 1: # 중부상권 row가 1줄 존재하는가?
    index_tmp = df_bid_tkyk_NF[df_bid_tkyk_NF['tkyk_name'] == '중부상권'].index[0] # 중부상권 인덱스번호
    sh_MS['D13'].options(index=False, header=False).value = df_bid_tkyk_NF.loc[index_tmp, 'i_cnt':'etc_cnt'].to_frame().T
    sh_MS['D14'].options(index=False, header=False).value = df_bid_tkyk_NF.loc[index_tmp, 'i_qty':'etc_qty'].to_frame().T
else: # 조건을 만족하지 않을 때는 0으로 처리한다.
    sh_MS['D13'].value = [0, 0, 0, 0, 0]
    sh_MS['D14'].value = [0, 0, 0, 0, 0]


# 대전상권
if len(df_bid_tkyk_NF[df_bid_tkyk_NF['tkyk_name'] == '대전상권'].index) == 1: # 대전상권 row가 1줄 존재하는가?
    index_tmp = df_bid_tkyk_NF[df_bid_tkyk_NF['tkyk_name'] == '대전상권'].index[0] # 대전상권 인덱스번호
    sh_MS['D22'].options(index=False, header=False).value = df_bid_tkyk_NF.loc[index_tmp, 'i_cnt':'etc_cnt'].to_frame().T
    sh_MS['D23'].options(index=False, header=False).value = df_bid_tkyk_NF.loc[index_tmp, 'i_qty':'etc_qty'].to_frame().T
else: # 조건을 만족하지 않을 때는 0으로 처리한다.
    sh_MS['D22'].value = [0, 0, 0, 0, 0]
    sh_MS['D23'].value = [0, 0, 0, 0, 0]

    
# 광주상권
if len(df_bid_tkyk_NF[df_bid_tkyk_NF['tkyk_name'] == '광주상권'].index) == 1: # 광주상권 row가 1줄 존재하는가?
    index_tmp = df_bid_tkyk_NF[df_bid_tkyk_NF['tkyk_name'] == '광주상권'].index[0] # 광주상권 인덱스번호
    sh_MS['D31'].options(index=False, header=False).value = df_bid_tkyk_NF.loc[index_tmp, 'i_cnt':'etc_cnt'].to_frame().T
    sh_MS['D32'].options(index=False, header=False).value = df_bid_tkyk_NF.loc[index_tmp, 'i_qty':'etc_qty'].to_frame().T
else: # 조건을 만족하지 않을 때는 0으로 처리한다.
    sh_MS['D31'].value = [0, 0, 0, 0, 0]
    sh_MS['D32'].value = [0, 0, 0, 0, 0]


# 대구상권
if len(df_bid_tkyk_NF[df_bid_tkyk_NF['tkyk_name'] == '대구상권'].index) == 1: # 대구상권 row가 1줄 존재하는가?
    index_tmp = df_bid_tkyk_NF[df_bid_tkyk_NF['tkyk_name'] == '대구상권'].index[0] # 대구상권 인덱스번호
    sh_MS['D40'].options(index=False, header=False).value = df_bid_tkyk_NF.loc[index_tmp, 'i_cnt':'etc_cnt'].to_frame().T
    sh_MS['D41'].options(index=False, header=False).value = df_bid_tkyk_NF.loc[index_tmp, 'i_qty':'etc_qty'].to_frame().T
else: # 조건을 만족하지 않을 때는 0으로 처리한다.
    sh_MS['D40'].value = [0, 0, 0, 0, 0]
    sh_MS['D41'].value = [0, 0, 0, 0, 0]


# 부산상권
if len(df_bid_tkyk_NF[df_bid_tkyk_NF['tkyk_name'] == '부산상권'].index) == 1: # 부산상권 row가 1줄 존재하는가?
    index_tmp = df_bid_tkyk_NF[df_bid_tkyk_NF['tkyk_name'] == '부산상권'].index[0] # 부산상권 인덱스번호
    sh_MS['D49'].options(index=False, header=False).value = df_bid_tkyk_NF.loc[index_tmp, 'i_cnt':'etc_cnt'].to_frame().T
    sh_MS['D50'].options(index=False, header=False).value = df_bid_tkyk_NF.loc[index_tmp, 'i_qty':'etc_qty'].to_frame().T
else: # 조건을 만족하지 않을 때는 0으로 처리한다.
    sh_MS['D49'].value = [0, 0, 0, 0, 0]
    sh_MS['D50'].value = [0, 0, 0, 0, 0]

index_tmp = '' # 초기화

#### F 시즌 -> M.S 시트에 기록

In [56]:
df_bid_tkyk_F

,quota,tkyk,sort,tkyk_name,i_cnt,s_cnt,e_cnt,l_cnt,etc_cnt,i_qty,s_qty,e_qty,l_qty,etc_qty
0,22F,C,1,서울상권,1,1,0,0,1,140,246,0,0,323
1,22F,H,2,중부상권,37,37,23,25,28,4762,5387,3353,4246,3790
2,22F,D,3,대전상권,0,0,0,0,0,0,0,0,0,0
3,22F,L,4,광주상권,48,50,38,48,59,6887,6812,5243,6556,8065
4,22F,I,5,대구상권,41,34,36,26,43,6426,5066,5709,4319,7857
5,22F,R,6,부산상권,23,7,10,8,16,2177,707,1225,1169,2818


In [57]:
# 서울상권
if len(df_bid_tkyk_F[df_bid_tkyk_F['tkyk_name'] == '서울상권'].index) == 1: # 서울상권 row가 1줄 존재하는가?
    index_tmp = df_bid_tkyk_F[df_bid_tkyk_F['tkyk_name'] == '서울상권'].index[0] # 서울상권 인덱스번호

    print(type(index_tmp))

    # 한 줄 추가할 때는 시리즈로 자동변환 되어서 강제로 데이터프레임 변경 후 축을 돌려서 입력함
    sh_MS['X4'].options(index=False, header=False).value = df_bid_tkyk_F.loc[index_tmp, 'i_cnt':'etc_cnt'].to_frame().T
    sh_MS['X5'].options(index=False, header=False).value = df_bid_tkyk_F.loc[index_tmp, 'i_qty':'etc_qty'].to_frame().T
else: # 조건을 만족하지 않을 때는 0으로 처리한다.
    sh_MS['X4'].value = [0, 0, 0, 0, 0]
    sh_MS['X5'].value = [0, 0, 0, 0, 0]


# 중부상권
if len(df_bid_tkyk_F[df_bid_tkyk_F['tkyk_name'] == '중부상권'].index) == 1: # 중부상권 row가 1줄 존재하는가?
    index_tmp = df_bid_tkyk_F[df_bid_tkyk_F['tkyk_name'] == '중부상권'].index[0] # 중부상권 인덱스번호
    sh_MS['X13'].options(index=False, header=False).value = df_bid_tkyk_F.loc[index_tmp, 'i_cnt':'etc_cnt'].to_frame().T
    sh_MS['X14'].options(index=False, header=False).value = df_bid_tkyk_F.loc[index_tmp, 'i_qty':'etc_qty'].to_frame().T
else: # 조건을 만족하지 않을 때는 0으로 처리한다.
    sh_MS['X13'].value = [0, 0, 0, 0, 0]
    sh_MS['X14'].value = [0, 0, 0, 0, 0]


# 대전상권
if len(df_bid_tkyk_F[df_bid_tkyk_F['tkyk_name'] == '대전상권'].index) == 1: # 대전상권 row가 1줄 존재하는가?
    index_tmp = df_bid_tkyk_F[df_bid_tkyk_F['tkyk_name'] == '대전상권'].index[0] # 대전상권 인덱스번호
    sh_MS['X22'].options(index=False, header=False).value = df_bid_tkyk_F.loc[index_tmp, 'i_cnt':'etc_cnt'].to_frame().T
    sh_MS['X23'].options(index=False, header=False).value = df_bid_tkyk_F.loc[index_tmp, 'i_qty':'etc_qty'].to_frame().T
else: # 조건을 만족하지 않을 때는 0으로 처리한다.
    sh_MS['X22'].value = [0, 0, 0, 0, 0]
    sh_MS['X23'].value = [0, 0, 0, 0, 0]

    
# 광주상권
if len(df_bid_tkyk_F[df_bid_tkyk_F['tkyk_name'] == '광주상권'].index) == 1: # 광주상권 row가 1줄 존재하는가?
    index_tmp = df_bid_tkyk_F[df_bid_tkyk_F['tkyk_name'] == '광주상권'].index[0] # 광주상권 인덱스번호
    sh_MS['X31'].options(index=False, header=False).value = df_bid_tkyk_F.loc[index_tmp, 'i_cnt':'etc_cnt'].to_frame().T
    sh_MS['X32'].options(index=False, header=False).value = df_bid_tkyk_F.loc[index_tmp, 'i_qty':'etc_qty'].to_frame().T
else: # 조건을 만족하지 않을 때는 0으로 처리한다.
    sh_MS['X31'].value = [0, 0, 0, 0, 0]
    sh_MS['X32'].value = [0, 0, 0, 0, 0]


# 대구상권
if len(df_bid_tkyk_F[df_bid_tkyk_F['tkyk_name'] == '대구상권'].index) == 1: # 대구상권 row가 1줄 존재하는가?
    index_tmp = df_bid_tkyk_F[df_bid_tkyk_F['tkyk_name'] == '대구상권'].index[0] # 대구상권 인덱스번호
    sh_MS['X40'].options(index=False, header=False).value = df_bid_tkyk_F.loc[index_tmp, 'i_cnt':'etc_cnt'].to_frame().T
    sh_MS['X41'].options(index=False, header=False).value = df_bid_tkyk_F.loc[index_tmp, 'i_qty':'etc_qty'].to_frame().T
else: # 조건을 만족하지 않을 때는 0으로 처리한다.
    sh_MS['X40'].value = [0, 0, 0, 0, 0]
    sh_MS['X41'].value = [0, 0, 0, 0, 0]


# 부산상권
if len(df_bid_tkyk_F[df_bid_tkyk_F['tkyk_name'] == '부산상권'].index) == 1: # 부산상권 row가 1줄 존재하는가?
    index_tmp = df_bid_tkyk_F[df_bid_tkyk_F['tkyk_name'] == '부산상권'].index[0] # 부산상권 인덱스번호
    sh_MS['X49'].options(index=False, header=False).value = df_bid_tkyk_F.loc[index_tmp, 'i_cnt':'etc_cnt'].to_frame().T
    sh_MS['X50'].options(index=False, header=False).value = df_bid_tkyk_F.loc[index_tmp, 'i_qty':'etc_qty'].to_frame().T
else: # 조건을 만족하지 않을 때는 0으로 처리한다.
    sh_MS['X49'].value = [0, 0, 0, 0, 0]
    sh_MS['X50'].value = [0, 0, 0, 0, 0]

index_tmp = '' # 초기화

<class 'numpy.int64'>


#### S 시즌 -> M.S 시트에 기록

In [58]:
df_bid_tkyk_S

,quota,tkyk,sort,tkyk_name,i_cnt,s_cnt,e_cnt,l_cnt,etc_cnt,i_qty,s_qty,e_qty,l_qty,etc_qty
0,23S,C,1,서울상권,4,6,4,6,9,745,1002,798,641,1599
1,23S,H,2,중부상권,10,15,6,12,24,2493,3521,1688,2293,5805
2,23S,D,3,대전상권,10,10,6,4,11,987,1609,1242,555,966
3,23S,L,4,광주상권,0,0,0,0,0,0,0,0,0,0
4,23S,I,5,대구상권,0,0,0,0,0,0,0,0,0,0
5,23S,R,6,부산상권,0,0,0,1,4,0,0,0,144,785


In [59]:
# 서울상권
if len(df_bid_tkyk_S[df_bid_tkyk_S['tkyk_name'] == '서울상권'].index) == 1: # 서울상권 row가 1줄 존재하는가?
    index_tmp = df_bid_tkyk_S[df_bid_tkyk_S['tkyk_name'] == '서울상권'].index[0] # 서울상권 인덱스번호

    # 한 줄 추가할 때는 시리즈로 자동변환 되어서 강제로 데이터프레임 변경 후 축을 돌려서 입력함
    sh_MS['N4'].options(index=False, header=False).value = df_bid_tkyk_S.loc[index_tmp, 'i_cnt':'etc_cnt'].to_frame().T
    sh_MS['N5'].options(index=False, header=False).value = df_bid_tkyk_S.loc[index_tmp, 'i_qty':'etc_qty'].to_frame().T
else: # 조건을 만족하지 않을 때는 0으로 처리한다.
    sh_MS['N4'].value = [0, 0, 0, 0, 0]
    sh_MS['N5'].value = [0, 0, 0, 0, 0]


# 중부상권
if len(df_bid_tkyk_S[df_bid_tkyk_S['tkyk_name'] == '중부상권'].index) == 1: # 중부상권 row가 1줄 존재하는가?
    index_tmp = df_bid_tkyk_S[df_bid_tkyk_S['tkyk_name'] == '중부상권'].index[0] # 중부상권 인덱스번호
    sh_MS['N13'].options(index=False, header=False).value = df_bid_tkyk_S.loc[index_tmp, 'i_cnt':'etc_cnt'].to_frame().T
    sh_MS['N14'].options(index=False, header=False).value = df_bid_tkyk_S.loc[index_tmp, 'i_qty':'etc_qty'].to_frame().T
else: # 조건을 만족하지 않을 때는 0으로 처리한다.
    sh_MS['N13'].value = [0, 0, 0, 0, 0]
    sh_MS['N14'].value = [0, 0, 0, 0, 0]


# 대전상권
if len(df_bid_tkyk_S[df_bid_tkyk_S['tkyk_name'] == '대전상권'].index) == 1: # 대전상권 row가 1줄 존재하는가?
    index_tmp = df_bid_tkyk_S[df_bid_tkyk_S['tkyk_name'] == '대전상권'].index[0] # 대전상권 인덱스번호
    sh_MS['N22'].options(index=False, header=False).value = df_bid_tkyk_S.loc[index_tmp, 'i_cnt':'etc_cnt'].to_frame().T
    sh_MS['N23'].options(index=False, header=False).value = df_bid_tkyk_S.loc[index_tmp, 'i_qty':'etc_qty'].to_frame().T
else: # 조건을 만족하지 않을 때는 0으로 처리한다.
    sh_MS['N22'].value = [0, 0, 0, 0, 0]
    sh_MS['N23'].value = [0, 0, 0, 0, 0]

    
# 광주상권
if len(df_bid_tkyk_S[df_bid_tkyk_S['tkyk_name'] == '광주상권'].index) == 1: # 광주상권 row가 1줄 존재하는가?
    index_tmp = df_bid_tkyk_S[df_bid_tkyk_S['tkyk_name'] == '광주상권'].index[0] # 광주상권 인덱스번호
    sh_MS['N31'].options(index=False, header=False).value = df_bid_tkyk_S.loc[index_tmp, 'i_cnt':'etc_cnt'].to_frame().T
    sh_MS['N32'].options(index=False, header=False).value = df_bid_tkyk_S.loc[index_tmp, 'i_qty':'etc_qty'].to_frame().T
else: # 조건을 만족하지 않을 때는 0으로 처리한다.
    sh_MS['N31'].value = [0, 0, 0, 0, 0]
    sh_MS['N32'].value = [0, 0, 0, 0, 0]


# 대구상권
if len(df_bid_tkyk_S[df_bid_tkyk_S['tkyk_name'] == '대구상권'].index) == 1: # 대구상권 row가 1줄 존재하는가?
    index_tmp = df_bid_tkyk_S[df_bid_tkyk_S['tkyk_name'] == '대구상권'].index[0] # 대구상권 인덱스번호
    sh_MS['N40'].options(index=False, header=False).value = df_bid_tkyk_S.loc[index_tmp, 'i_cnt':'etc_cnt'].to_frame().T
    sh_MS['N41'].options(index=False, header=False).value = df_bid_tkyk_S.loc[index_tmp, 'i_qty':'etc_qty'].to_frame().T
else: # 조건을 만족하지 않을 때는 0으로 처리한다.
    sh_MS['N40'].value = [0, 0, 0, 0, 0]
    sh_MS['N41'].value = [0, 0, 0, 0, 0]


# 부산상권
if len(df_bid_tkyk_S[df_bid_tkyk_S['tkyk_name'] == '부산상권'].index) == 1: # 부산상권 row가 1줄 존재하는가?
    index_tmp = df_bid_tkyk_S[df_bid_tkyk_S['tkyk_name'] == '부산상권'].index[0] # 부산상권 인덱스번호
    sh_MS['N49'].options(index=False, header=False).value = df_bid_tkyk_S.loc[index_tmp, 'i_cnt':'etc_cnt'].to_frame().T
    sh_MS['N50'].options(index=False, header=False).value = df_bid_tkyk_S.loc[index_tmp, 'i_qty':'etc_qty'].to_frame().T
else: # 조건을 만족하지 않을 때는 0으로 처리한다.
    sh_MS['N49'].value = [0, 0, 0, 0, 0]
    sh_MS['N50'].value = [0, 0, 0, 0, 0]

index_tmp = '' # 초기화

### '일일보고.xlsx 파일 오픈

In [60]:
try:
    wb_day = xw.Book(f"./'일일보고.xlsx") # type: xlwings.main.Book
    sh_yesterday = wb_day.sheets[-1] # type: xlwings.main.Sheet
except FileNotFoundError:
    print("같은 폴더안에 \n'일일보고.xlsx \n파일이 없습니다.\n\n프로그램을 종료합니다.")
    pyautogui.alert(title='FileNotFoundError', text="같은 폴더안에 \n'일일보고.xlsx \n파일이 없습니다.\n\n프로그램을 종료합니다.")
    sys.exit()

#### 마지막 시트복사

- copy (before=None, after=None, name=None)
- 매개변수 :
    - before ( sheet object , default None ) – 시트를 배치하려는 시트 개체 앞에
    - after ( sheet object , default None ) – 시트를 배치하려는 뒤에 오는 시트 객체, 기본적으로 모든 기존 시트 뒤에 배치됩니다.
    - name ( str , 기본값 None ) – 복사본의 시트 이름

In [61]:
try:
    sh_yesterday.copy(name=report_date[2:])
except ValueError:
    print(f'작성할 {report_date[2:]} 시트명이 이미 존재합니다.\n해당 시트를 삭제 후 프로그램을 다시 실행하세요.')
    pyautogui.alert(title='ValueError', text=f'작성할 {report_date[2:]} 시트명이 이미 존재합니다.\n해당 시트를 삭제 후 프로그램을 다시 실행하세요.')
    sys.exit()

#### 오늘 작성할 시트 지정 (복사된)

In [62]:
sh_today = wb_day.sheets[report_date[2:]] # type: xlwings.main.Sheet

#### 제목 -> 오늘 날짜

##### A1

In [63]:
title_sh: str = report_date[:4] + '.' + report_date[4:6] + '.' + report_date[6:] + ' 영업팀 일일보고'
sh_today['A1'].value = title_sh

#### 어제 데이터 영역 복사

##### F3

In [64]:
sh_today['F3'].value = sh_today['B3:E21'].value
sh_today['B3:E21'].value

[[datetime.datetime(2022, 8, 8, 0, 0), None, None, None],
 ['22F+23N', "'23N", "'23S", "'22F"],
 [174.0, 24.0, 22.0, 150.0],
 [24928.0, 4536.0, 4065.0, 20392.0],
 [0.206, 0.163, 0.153, 0.219],
 [160.0, 31.0, 31.0, 129.0],
 [24400.0, 6182.0, 6182.0, 18218.0],
 [0.201, 0.222, 0.232, 0.195],
 [123.0, 16.0, 15.0, 107.0],
 [19287.0, 3757.0, 3557.0, 15530.0],
 [0.159, 0.135, 0.134, 0.166],
 [131.0, 24.0, 23.0, 107.0],
 [20179.0, 3889.0, 3633.0, 16290.0],
 [0.167, 0.139, 0.137, 0.175],
 [195.0, 48.0, 48.0, 147.0],
 [32394.0, 9541.0, 9155.0, 22853.0],
 [0.267, 0.342, 0.344, 0.245],
 [783.0, 143.0, 139.0, 640.0],
 [121188.0, 27905.0, 26592.0, 93283.0]]

#### 오늘 데이터 영역 복사 (M.S파일에서) ***

- 복사하지 않고 직접 뽑아옴

##### B5

In [65]:
sh_today['B5'].options(index=False, header=False).value = df_ms_total
df_ms_total

quota,23N22F,23N,23S,22F
0,176.000,26.000,24.000,150.000
1,25088.000,4696.000,4225.000,20392.000
2,0.207,0.167,0.157,0.219
3,160.000,31.000,31.000,129.000
4,24350.000,6132.000,6132.000,18218.000
5,0.200,0.218,0.228,0.195
6,124.000,17.000,16.000,107.000
7,19458.000,3928.000,3728.000,15530.000
8,0.160,0.139,0.139,0.166
9,131.000,24.000,23.000,107.000


#### 오늘 날짜 입력

##### B3

In [66]:
sh_today['B3'].value = report_date2
report_date2

'2022-08-09'

#### # 동복 -> N + F 시즌 낙찰현황 복사

##### B26

In [67]:
sh_today['B26'].options(index=False, header=False).value = df_bid_tkyk_NF_ISEL_sum
df_bid_tkyk_NF_ISEL_sum

,i_cnt,i_qty,s_cnt,s_qty,e_cnt,e_qty,l_cnt,l_qty,sum_cnt,sum_qty
0,5,885,7,1248,4,798,6,641,22,3572
1,47,7255,52,8908,29,5041,37,6539,165,27743
2,10,987,10,1609,6,1242,4,555,30,4393
3,50,7358,50,6812,39,5443,49,6812,188,26425
4,41,6426,34,5066,36,5709,26,4319,137,21520
5,23,2177,7,707,10,1225,9,1313,49,5422
6,176,25088,160,24350,124,19458,131,20179,591,89075


#### # 하복 -> S 시즌 낙찰현황 복사

##### B48

In [68]:
sh_today['B48'].options(index=False, header=False).value = df_bid_tkyk_S_ISEL_sum
df_bid_tkyk_S_ISEL_sum

,i_cnt,i_qty,s_cnt,s_qty,e_cnt,e_qty,l_cnt,l_qty,sum_cnt,sum_qty
0,4,745,6,1002,4,798,6,641,20,3186
1,10,2493,15,3521,6,1688,12,2293,43,9995
2,10,987,10,1609,6,1242,4,555,30,4393
3,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,1,144,1,144
6,24,4225,31,6132,16,3728,23,3633,94,17718


#### # F 시즌 -> F 시즌 낙찰현황 복사

##### B59

In [69]:
sh_today['B59'].options(index=False, header=False).value = df_bid_tkyk_F_ISEL_sum
df_bid_tkyk_F_ISEL_sum

,i_cnt,i_qty,s_cnt,s_qty,e_cnt,e_qty,l_cnt,l_qty
0,1,140,1,246,0,0,0,0
1,37,4762,37,5387,23,3353,25,4246
2,0,0,0,0,0,0,0,0
3,48,6887,50,6812,38,5243,48,6556
4,41,6426,34,5066,36,5709,26,4319
5,23,2177,7,707,10,1225,8,1169
6,150,20392,129,18218,107,15530,107,16290


## 데이터 프레임 작성

### 기록할 포지션 잡는 함수 (상대좌표)

In [70]:
set_position: list = [] # 가변 좌표

def set_pos(sheet) -> None:

    # 범위 안에 원하는 TEXT 값 찾기
    i: int = 0

    for row in range(200, 1, -1): # row 범위
        for col in range(1, 2): # column 범위
            if sheet.range((row, col)).value == "서울":
                i += 1
                if i < 4: # 뒤에서부터 1, 2, 3번째를 찾는다.
                    print(f'{col}열 {row}행')
                    set_position.append(row)
                    print(set_position)

    if len(set_position) != 3:
        print(f"기준좌표 갯수가 3 이어야 합니다.\n현재값 : {len(set_position)}\n\n프로그램을 종료합니다.")
        pyautogui.alert(title='논리 에러', text=f"기준좌표 갯수가 3 이어야 합니다.\n현재값 : {len(set_position)}\n\n프로그램을 종료합니다.")
        sys.exit()

set_pos(sh_today)
set_position.sort() # 오름차순 정렬
set_position

1열 114행
[114]
1열 102행
[114, 102]
1열 89행
[114, 102, 89]


[89, 102, 114]

#### # 수주 전일데이터 복사

##### E90, E103, E115

In [71]:
sh_today[f'E{set_position[0]}'].value = sh_today[f'B{set_position[0]}:D{set_position[0]+6}'].value # 동복(F + N) 수주 현황 ( J + H 기준 )
sh_today[f'E{set_position[1]}'].value = sh_today[f'B{set_position[1]}:D{set_position[1]+6}'].value # 하복(S) 수주 현황 ( 하의기준 )
sh_today[f'E{set_position[2]}'].value = sh_today[f'B{set_position[2]}:D{set_position[2]+6}'].value # 동복(F) 수주 현황 ( J + H 기준 )

### 시즌, 상권별 수주/해제량

#### NF, 수주량/해제량, J+H

In [72]:
# 특약 채우기 함수 2

def add_tkyk2(df: pd.DataFrame) -> pd.DataFrame:
    df = df.reset_index()
    tkyk_lost: list = list(set(tkyk_sort) - set(df['sort'].tolist())) # 세트 변환 후, 차집합(없는 특약) = 모든 특약 - 현재 특약, 다시 리스트 변환
    
    for t in tkyk_lost:
        tkyk_insert: list = [] # 빈 리스트 생성
        tkyk_insert.append(tkyk_sort[t-1]) # sort 용 컬럼
        tkyk_insert.append(tkyk_code[t-1]) # 특약코드
        tkyk_insert.append(0) # 0으로 채운다
        tkyk_insert.append(0) # 0으로 채운다
        df.loc[len(df)] = tkyk_insert # 데이터프레임의 마지막 행에 넣는다.
    
    df = df.sort_values('sort').reset_index(drop=True) # 특약 정렬
    return df

In [73]:
cond1 = df_suju_qty['quota'] == N_season + F_season
cond2 = df_suju_qty['gubun'] == '수주량'
cond3 = df_suju_qty['gubun'] == '해제량'

df_NF_JH_SUJU: pd.DataFrame = df_suju_qty[cond1 & cond2]
df_NF_JH_HAJE: pd.DataFrame = df_suju_qty[cond1 & cond3]

df_NF_JH_SUJU, df_NF_JH_HAJE

(     quota gubun bokjong tkyk tkyk_name  sort   qty
 20  23N22F   수주량       J    C      서울상권     1   385
 21  23N22F   수주량       J    H      중부상권     2  6893
 22  23N22F   수주량       J    D      대전상권     3  5326
 23  23N22F   수주량       J    L      광주상권     4  5436
 24  23N22F   수주량       J    I      대구상권     5  5770
 25  23N22F   수주량       J    R      부산상권     6  5341
 26  23N22F   수주량       H    C      서울상권     1   480
 27  23N22F   수주량       H    H      중부상권     2    67
 28  23N22F   수주량       H    D      대전상권     3  1386
 29  23N22F   수주량       H    L      광주상권     4  2530
 30  23N22F   수주량       H    I      대구상권     5  1546,
      quota gubun bokjong tkyk tkyk_name  sort   qty
 31  23N22F   해제량       J    C      서울상권     1   295
 32  23N22F   해제량       J    H      중부상권     2  5139
 33  23N22F   해제량       J    D      대전상권     3   826
 34  23N22F   해제량       J    L      광주상권     4  3540
 35  23N22F   해제량       J    I      대구상권     5  4788
 36  23N22F   해제량       J    R      부산상권     

In [74]:
df_NF_JH: pd.DataFrame = pd.concat([df_NF_JH_SUJU.groupby(['sort', 'tkyk'])['qty'].agg(sum),
                                    df_NF_JH_HAJE.groupby(['sort', 'tkyk'])['qty'].agg(sum)
                                   ], axis=1)
df_NF_JH.columns = ['수주량', '해제량']

df_NF_JH = add_tkyk2(df_NF_JH.copy()) # 특약별 집계량 없을 때 빈 라인 채워주는 함수

df_NF_JH.loc[len(df_NF_JH)] = df_NF_JH.sum(axis=0) # 합계

df_NF_JH['해제율'] = round(df_NF_JH['해제량'] / df_NF_JH['수주량'], 3)

df_NF_JH = df_NF_JH.fillna(0) # 해제량이 없으면 NaN 값이 있을 수 있다. NaN 값이 발생할 수 있는 최종 위치

df_NF_JH['수주량'] = df_NF_JH['수주량'].astype(int) # int 변환은 NaN이 없어야 가능
df_NF_JH['해제량'] = df_NF_JH['해제량'].astype(int)

df_NF_JH = df_NF_JH.set_index(['sort', 'tkyk']) # 인덱스 처리

df_NF_JH

,,수주량,해제량,해제율
sort,tkyk,,,
1,C,865,445,0.514
2,H,6960,5206,0.748
3,D,6712,904,0.135
4,L,7966,5637,0.708
5,I,7316,5869,0.802
6,R,5341,1929,0.361
21,CHDLIR,35160,19990,0.569


##### 포지션 1

In [75]:
sh_today[f'B{set_position[0]}'].options(index=False, header=False).value = df_NF_JH
df_NF_JH

,,수주량,해제량,해제율
sort,tkyk,,,
1,C,865,445,0.514
2,H,6960,5206,0.748
3,D,6712,904,0.135
4,L,7966,5637,0.708
5,I,7316,5869,0.802
6,R,5341,1929,0.361
21,CHDLIR,35160,19990,0.569


#### S, 수주량/해제량, 하의

In [76]:
cond1 = df_suju_qty['quota'] == S_season
cond2 = df_suju_qty['gubun'] == '수주량'
cond3 = df_suju_qty['gubun'] == '해제량'

df_S_HAI_SUJU = df_suju_qty[cond1 & cond2]
df_S_HAI_HAJE = df_suju_qty[cond1 & cond3]

df_S_HAI_SUJU, df_S_HAI_HAJE

(   quota gubun bokjong tkyk tkyk_name  sort   qty
 42   23S   수주량      HA    C      서울상권     1   704
 43   23S   수주량      HA    H      중부상권     2  2172
 44   23S   수주량      HA    D      대전상권     3   716
 45   23S   수주량      HA    L      광주상권     4   163
 46   23S   수주량      HA    I      대구상권     5   232
 47   23S   수주량      HA    R      부산상권     6    90,
    quota gubun bokjong tkyk tkyk_name  sort   qty
 48   23S   해제량      HA    C      서울상권     1    78
 49   23S   해제량      HA    H      중부상권     2  1242
 50   23S   해제량      HA    D      대전상권     3   320
 51   23S   해제량      HA    L      광주상권     4   163
 52   23S   해제량      HA    I      대구상권     5   232)

In [77]:
df_S_HAI = pd.concat([df_S_HAI_SUJU.groupby(['sort', 'tkyk'])['qty'].agg(sum),
                      df_S_HAI_HAJE.groupby(['sort', 'tkyk'])['qty'].agg(sum)
                     ], axis=1)
df_S_HAI.columns = ['수주량', '해제량']

df_S_HAI = add_tkyk2(df_S_HAI.copy()) # 특약별 집계량 없을 때 빈 라인 채워주는 함수

df_S_HAI.loc[len(df_S_HAI)] = df_S_HAI.sum(axis=0) # 합계

df_S_HAI['해제율'] = round(df_S_HAI['해제량'] / df_S_HAI['수주량'], 3)

df_S_HAI = df_S_HAI.fillna(0) # 해제량이 없으면 NaN 값이 있을 수 있다. NaN 값이 발생할 수 있는 최종 위치

df_S_HAI['수주량'] = df_S_HAI['수주량'].astype(int) # int 변환은 NaN이 없어야 가능
df_S_HAI['해제량'] = df_S_HAI['해제량'].astype(int)

df_S_HAI = df_S_HAI.set_index(['sort', 'tkyk']) # 인덱스 처리

df_S_HAI

,,수주량,해제량,해제율
sort,tkyk,,,
1,C,704,78,0.111
2,H,2172,1242,0.572
3,D,716,320,0.447
4,L,163,163,1.000
5,I,232,232,1.000
6,R,90,0,0.000
21,CHDLIR,4077,2035,0.499


##### 포지션 2

In [78]:
sh_today[f'B{set_position[1]}'].options(index=False, header=False).value = df_S_HAI
df_S_HAI

,,수주량,해제량,해제율
sort,tkyk,,,
1,C,704,78,0.111
2,H,2172,1242,0.572
3,D,716,320,0.447
4,L,163,163,1.000
5,I,232,232,1.000
6,R,90,0,0.000
21,CHDLIR,4077,2035,0.499


#### F, 수주량/해제량, J+H

In [79]:
cond1 = df_suju_qty['quota'] == F_season
cond2 = df_suju_qty['gubun'] == '수주량'
cond3 = df_suju_qty['gubun'] == '해제량'

df_F_JH_SUJU: pd.DataFrame = df_suju_qty[cond1 & cond2]
df_F_JH_HAJE: pd.DataFrame = df_suju_qty[cond1 & cond3]

df_F_JH_SUJU, df_F_JH_HAJE

(  quota gubun bokjong tkyk tkyk_name  sort   qty
 0   22F   수주량       J    C      서울상권     1   177
 1   22F   수주량       J    H      중부상권     2  4385
 2   22F   수주량       J    D      대전상권     3    54
 3   22F   수주량       J    L      광주상권     4  3528
 4   22F   수주량       J    I      대구상권     5  4688
 5   22F   수주량       J    R      부산상권     6  1777
 6   22F   수주량       H    H      중부상권     2    67
 7   22F   수주량       H    D      대전상권     3    10
 8   22F   수주량       H    L      광주상권     4  2530
 9   22F   수주량       H    I      대구상권     5  1546,
    quota gubun bokjong tkyk tkyk_name  sort   qty
 10   22F   해제량       J    C      서울상권     1   177
 11   22F   해제량       J    H      중부상권     2  3805
 12   22F   해제량       J    D      대전상권     3    54
 13   22F   해제량       J    L      광주상권     4  3306
 14   22F   해제량       J    I      대구상권     5  4052
 15   22F   해제량       J    R      부산상권     6  1747
 16   22F   해제량       H    H      중부상권     2    67
 17   22F   해제량       H    D      대전상권   

In [80]:
df_F_JH: pd.DataFrame = pd.concat([df_F_JH_SUJU.groupby(['sort', 'tkyk'])['qty'].agg(sum),
                                   df_F_JH_HAJE.groupby(['sort', 'tkyk'])['qty'].agg(sum)
                                  ], axis=1)
df_F_JH.columns = ['수주량', '해제량']

df_F_JH = add_tkyk2(df_F_JH.copy()) # 특약별 집계량 없을 때 빈 라인 채워주는 함수

df_F_JH.loc[len(df_F_JH)] = df_F_JH.sum(axis=0) # 합계

df_F_JH['해제율'] = round(df_F_JH['해제량'] / df_F_JH['수주량'], 3)

df_F_JH = df_F_JH.fillna(0) # 해제량이 없으면 NaN 값이 있을 수 있다. NaN 값이 발생할 수 있는 최종 위치

df_F_JH['수주량'] = df_F_JH['수주량'].astype(int) # int 변환은 NaN이 없어야 가능
df_F_JH['해제량'] = df_F_JH['해제량'].astype(int)

df_F_JH = df_F_JH.set_index(['sort', 'tkyk']) # 인덱스 처리

df_F_JH

,,수주량,해제량,해제율
sort,tkyk,,,
1,C,177,177,1.000
2,H,4452,3872,0.870
3,D,64,64,1.000
4,L,6058,5403,0.892
5,I,6234,5133,0.823
6,R,1777,1747,0.983
21,CHDLIR,18762,16396,0.874


##### 포지션 3

In [81]:
sh_today[f'B{set_position[2]}'].options(index=False, header=False).value = df_F_JH
df_F_JH

,,수주량,해제량,해제율
sort,tkyk,,,
1,C,177,177,1.000
2,H,4452,3872,0.870
3,D,64,64,1.000
4,L,6058,5403,0.892
5,I,6234,5133,0.823
6,R,1777,1747,0.983
21,CHDLIR,18762,16396,0.874


#### 시즌, 수주/해제, 복종별 집계

- df.groupby([열1,열2...열n])[[열1,열2...열n]].agg("집계함수").reset_index()
- df.pivot_table(values열, index=[열1,열2...열n], columns=[열1,열2...열n], aggfunc="집계함수")

In [82]:
df_suju_qty_sum: pd.DataFrame = df_suju_qty.groupby(['quota', 'gubun', 'bokjong'])['qty'].agg(sum).to_frame()
df_suju_qty_sum = df_suju_qty_sum.reset_index()
df_suju_qty_sum

,quota,gubun,bokjong,qty
0,22F,수주량,H,4153
1,22F,수주량,J,14609
2,22F,해제량,H,3255
3,22F,해제량,J,13141
4,23N22F,수주량,H,6009
5,23N22F,수주량,J,29151
6,23N22F,해제량,H,3473
7,23N22F,해제량,J,16517
8,23S,수주량,HA,4077
9,23S,해제량,HA,2035


In [83]:
cond1_1 = df_suju_qty_sum['quota'] == N_season + F_season
# cond1_2 = df_suju_qty_sum['quota'] == S_season
cond1_3 = df_suju_qty_sum['quota'] == F_season
cond2_1 = df_suju_qty_sum['gubun'] == '수주량'
cond2_2 = df_suju_qty_sum['gubun'] == '해제량'
cond3_1 = df_suju_qty_sum['bokjong'] == 'H'
cond3_2 = df_suju_qty_sum['bokjong'] == 'J'
# cond3_3 = df_suju_qty_sum['bokjong'] == sum_bokjong[2]

# NF
NF_SUJU_J = int(df_suju_qty_sum['qty'][cond1_1 & cond2_1 & cond3_1].sum())
NF_SUJU_H = int(df_suju_qty_sum['qty'][cond1_1 & cond2_1 & cond3_2].sum())
NF_HAJE_J = int(df_suju_qty_sum['qty'][cond1_1 & cond2_2 & cond3_1].sum())
NF_HAJE_H = int(df_suju_qty_sum['qty'][cond1_1 & cond2_2 & cond3_2].sum())

# S (사용안함)
# S_SUJU_HA = int(df_suju_qty_sum['qty'][cond1_2 & cond2_1 & cond3_3].sum())
# S_HAJE_HA = int(df_suju_qty_sum['qty'][cond1_2 & cond2_2 & cond3_3].sum())

# F
F_SUJU_J = int(df_suju_qty_sum['qty'][cond1_3 & cond2_1 & cond3_1].sum())
F_SUJU_H = int(df_suju_qty_sum['qty'][cond1_3 & cond2_1 & cond3_2].sum())
F_HAJE_J = int(df_suju_qty_sum['qty'][cond1_3 & cond2_2 & cond3_1].sum())
F_HAJE_H = int(df_suju_qty_sum['qty'][cond1_3 & cond2_2 & cond3_2].sum())

NF_SUJU_J, NF_SUJU_H, NF_HAJE_J, NF_HAJE_H, F_SUJU_J, F_SUJU_H, F_HAJE_J, F_HAJE_H

(6009, 29151, 3473, 16517, 4153, 14609, 3255, 13141)

##### (포지션 1) + 7

In [84]:
sh_today[f'A{set_position[0]+7}'].value = f'수주량 J : {NF_SUJU_J:,} ,  H : {NF_SUJU_H:,} / 해제량 J : {NF_HAJE_J:,} ,  H : {NF_HAJE_H:,}'
print(f'수주량 J : {NF_SUJU_J:,} ,  H : {NF_SUJU_H:,} / 해제량 J : {NF_HAJE_J:,} ,  H : {NF_HAJE_H:,}')

수주량 J : 6,009 ,  H : 29,151 / 해제량 J : 3,473 ,  H : 16,517


##### (포지션 3) + 7

In [85]:
sh_today[f'A{set_position[2]+7}'].value = f'수주량 J : {F_SUJU_J:,} ,  H : {F_SUJU_H:,} / 해제량 J : {F_HAJE_J:,} ,  H : {F_HAJE_H:,}'
print(f'수주량 J : {F_SUJU_J:,} ,  H : {F_SUJU_H:,} / 해제량 J : {F_HAJE_J:,} ,  H : {F_HAJE_H:,}')

수주량 J : 4,153 ,  H : 14,609 / 해제량 J : 3,255 ,  H : 13,141


#### 전주간 수주/해제량

In [86]:
cond1_1: pd.Series = df_suju_qty_week['quota'] == N_season + F_season
cond1_2: pd.Series = df_suju_qty_week['quota'] == S_season
cond1_3: pd.Series = df_suju_qty_week['quota'] == F_season
cond2_1: pd.Series = df_suju_qty_week['gubun'] == '수주량'
cond2_2: pd.Series = df_suju_qty_week['gubun'] == '해제량'
cond3_1: pd.Series = df_suju_qty_week['bokjong'] == 'J'
cond3_2: pd.Series = df_suju_qty_week['bokjong'] == 'HA'

# NF
NF_SUJU_J_WEEK = int(df_suju_qty_week['qty'][cond1_1 & cond2_1 & cond3_1].sum())
NF_HAJE_J_WEEK = int(df_suju_qty_week['qty'][cond1_1 & cond2_2 & cond3_1].sum())

# S
S_SUJU_HA_WEEK = int(df_suju_qty_week['qty'][cond1_2 & cond2_1 & cond3_2].sum())
S_HAJE_HA_WEEK = int(df_suju_qty_week['qty'][cond1_2 & cond2_2 & cond3_2].sum())

# F
F_SUJU_J_WEEK = int(df_suju_qty_week['qty'][cond1_3 & cond2_1 & cond3_1].sum())
F_HAJE_J_WEEK = int(df_suju_qty_week['qty'][cond1_3 & cond2_2 & cond3_1].sum())

NF_SUJU_J_WEEK, NF_HAJE_J_WEEK, S_SUJU_HA_WEEK, S_HAJE_HA_WEEK, F_SUJU_J_WEEK, F_HAJE_J_WEEK

(0, 220, 0, 0, 0, 102)

In [87]:
NF_SUJU_J_WEEK

0

##### (포지션 1-3) + 7-9

In [88]:
sh_today[f'A{set_position[0]+8}'].value = f'前주간 J 수주량 : {NF_SUJU_J_WEEK:,}'
sh_today[f'A{set_position[0]+9}'].value = f'前주간 J 해제량 : {NF_HAJE_J_WEEK:,}'

sh_today[f'A{set_position[1]+7}'].value = f'前주간 수주량 : {S_SUJU_HA_WEEK:,}'
sh_today[f'A{set_position[1]+8}'].value = f'前주간 해제량 : {S_HAJE_HA_WEEK:,}'

sh_today[f'A{set_position[2]+8}'].value = f'前주간 J 수주량 : {F_SUJU_J_WEEK:,}'
sh_today[f'A{set_position[2]+9}'].value = f'前주간 J 해제량 : {F_HAJE_J_WEEK:,}'

print(f'前주간 J 수주량 : {NF_SUJU_J_WEEK:,}')
print(f'前주간 J 해제량 : {NF_HAJE_J_WEEK:,}')
print(f'前주간 수주량 : {S_SUJU_HA_WEEK:,}')
print(f'前주간 해제량 : {S_HAJE_HA_WEEK:,}')
print(f'前주간 J 수주량 : {F_SUJU_J_WEEK:,}')
print(f'前주간 J 해제량 : {F_HAJE_J_WEEK:,}')

前주간 J 수주량 : 0
前주간 J 해제량 : 220
前주간 수주량 : 0
前주간 해제량 : 0
前주간 J 수주량 : 0
前주간 J 해제량 : 102


### 파일저장 및 종료

In [89]:
wb.save()

In [90]:
wb.close()

In [91]:
wb_day.save()

In [92]:
wb_day.close()

## 작업종료

In [93]:
app.kill()

### 수행시간측정 종료

In [94]:
end_lab: float = timer()

In [95]:
pyautogui.alert(title='수행시간 측정', text=f'시작시각 : {start_lab}\n종료시각 : {end_lab}\n\n총 수행시간 : {end_lab-start_lab} 초')

'OK'

In [96]:
pyautogui.alert(title='주의사항', text='수치데이터 이외에 코멘트 기입은 수기로 작성해주시기 바랍니다.\n엑셀양식 표 위치를 변경하면 정상동작하지 않습니다.\n표 위치를 변경하면 프로그램을 수정해야 합니다.')

'OK'

In [97]:
pyautogui.alert(title='프로그램 완료', text='지루한 작업이 모두 끝났습니다. \n수고하셨습니다.')

'OK'